# Giriş

Bu çalışmada n sayıda işin m sayıda makineye atandığı atölye çizelgeleme problemi çalışılmıştır. Çalışmadaki işlerin birden fazla rotası bulunmaktadır. İşler rotaları seçildikten sonra çizelgelemeye alınırlar. İşlere ait teslim tarihi ataması belirlenen kurallar içerisinden seçilerek yapılır. 

Bu kısımda kullanılacak kütüphaneler probleme dahil edilmiştir.

In [1]:
# Imports
import numpy as np
import pandas as pd
from copy import deepcopy
from math import ceil, log, cos, exp, fabs
import salabim as sim
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as col
from pprint import pprint
import time
#import glob
from cycler import cycler
import codecs  # dosyaları utf8 kodlamasında kaydetmek için
from scipy.stats import truncnorm  # normal dağılıma uyan sayı üretmek için
import pickle
import json
# SF Conf
ddrule_size = 36
dsprule_size = 23

SEED = 12345
#SEED = int(input("Please enter random number generation seed: "))

np.random.seed(SEED)


class ProblemSet:
    DATA = [[4, 25, 50, 75, 100, 125, 150, 175, 200],  # 0. Is Sayilari
            [4, 10, 10, 10, 10, 10, 10, 10, 10],  # 1. Operasyon Sayilari
            [2, 5, 10, 15, 20, 25, 30, 35, 40],  # 2. Makine Sayilari
            [2, 5, 5, 5, 5, 3, 3, 3, 3],  # 3. Rota Sayilari
            [10, 150, 150, 100, 100, 75, 75, 50, 5]]  # iter_size

    def __init__(self, ps_id):
        self.ps_id = ps_id
        self.job_size = ProblemSet.DATA[0][ps_id]
        self.operation_size = ProblemSet.DATA[1][ps_id]
        self.machine_size = ProblemSet.DATA[2][ps_id]
        self.route_size = ProblemSet.DATA[3][ps_id]
        self.iter_size = ProblemSet.DATA[4][ps_id]


def sum_to_x(n, x):
    """gives n random value whose sum is equal to x"""
    values = [0.0, x] + list(np.random.uniform(low=0.0, high=x, size=n - 1))
    values.sort()
    return [values[i + 1] - values[i] for i in range(n)]


def exec_time(start, end):
    diff_time = end - start
    m, s = divmod(diff_time, 60)
    h, m = divmod(m, 60)
    s, m, h = int(round(s, 0)), int(round(m, 0)), int(round(h, 0))
    print("Execution Time: " + "{0:02d}:{1:02d}:{2:02d}".format(h, m, s))
    return [h, m, s]


def make_second(liste):
    total_seconds = 0
    total_seconds += liste[2]+60*liste[1]+60*60*liste[0]
    return total_seconds


# mean ortalama, sd standart sapma, low upp alt ve üst limit
def get_truncated_normal(mean=0, sd=1, low=0, upp=10):
    """https://stackoverflow.com/questions/36894191/how-to-get-a-normal-distribution-within-a-range-in-numpy"""
    return truncnorm(
        (low - mean) / sd, (upp - mean) / sd, loc=mean, scale=sd)


def writeToJSONFile(path, fileName, data):
    filePathNameWExt = './' + path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w') as fp:
        json.dump(data, fp)

In [ ]:
# inputs for the shop floor
np.random.seed(SEED)
# generate arrival times
number_of_jobs = [4, 25, 50, 75, 100, 125, 150, 175, 200]
weight_list0 = [0.33, 0.66, 1]
weight_list1 = [0.25, 0.50, 0.75, 1]
weight_list2 = [0.2, 0.4, 0.6, 0.8, 1]
weight_list3 = [0.16, 0.32, 0.48, 0.64, 0.8, 1]

for i in range(9):
    ps = ProblemSet(i)
    machine_numbers = np.random.randint(0, ps.machine_size, size=(
        ps.job_size, ps.route_size, ps.operation_size))
    operation_durations = abs(np.random.normal(
        6, 12, (ps.job_size, ps.route_size, ps.operation_size))).astype(int)+1
    with open('inputs\\machine_numbers_'+str(i)+'.txt', 'w') as outfile:
        outfile.write('# Job size: {}, Route size {}, Operation size {} \n'.format(
            ps.job_size, ps.route_size, ps.operation_size))
        outfile.write('# Machine size: {}\n'.format(ps.machine_size))
        for data_slice in machine_numbers:
            np.savetxt(outfile, X=data_slice, fmt='%i')
            outfile.write('# New job\n')
    with open('inputs\\operation_durations_'+str(i)+'.txt', 'w') as outfile:
        outfile.write('# Job size: {}, Route size {}, Operation size {} \n'.format(
            ps.job_size, ps.route_size, ps.operation_size))
        outfile.write('# Machine size: {}\n'.format(ps.machine_size))
        for data_slice in operation_durations:
            np.savetxt(outfile, X=data_slice, fmt='%i')
            outfile.write('# New job\n')
    arrival_times = np.cumsum(
        list(map(int, np.random.exponential(20, number_of_jobs[i]))))
    np.savetxt('inputs\\arrivals_'+str(i)+'.txt', X=arrival_times, fmt='%i')
    if 0 <= i < 3:
        w_l = weight_list0
    elif 3 <= i < 5:
        w_l = weight_list1
    elif 5 <= i < 7:
        w_l = weight_list2
    elif 7 <= i < 9:
        w_l = weight_list3
    weights = np.random.choice(w_l, size=number_of_jobs[i], replace=True)
    np.savetxt('inputs\\weights_'+str(i)+'.txt', X=weights, fmt='%1.3f')

# Atölye Numarası

In [2]:
# Choose Shop Floor
at_no = int(input("Enter shop floor: "))
ps = ProblemSet(at_no)

machine_numbers = np.loadtxt(fname='inputs\\machine_numbers_'+str(
    at_no)+'.txt', dtype=int).reshape(ps.job_size, ps.route_size, ps.operation_size)

operation_durations = np.loadtxt(fname='inputs\\operation_durations_'+str(
    at_no)+'.txt', dtype=int).reshape(ps.job_size, ps.route_size, ps.operation_size)

weights = np.loadtxt('inputs/weights_'+str(at_no)+'.txt', dtype=float)

arrival_times = np.loadtxt('inputs/arrivals_'+str(at_no)+'.txt', dtype=int)

Enter shop floor: 0


# Sınıflar
İş
self.job_id = job_id 
self.name('job' + str(job_id))
self.chromosome = chromosome
self.operations = sim.Queue(name=self.name() + '.operations')
self.operation_list = []
self.arrival_time = arrival_times[job_id]
self.weight = np.random.choice(list_weights, size=1, replace=True)[0]
self.due_date_time = -1
self.departure_time = -1
self.color = None

Teslim tarihi belirleme
pav: seçilen rotaya göre işlerin operasyon sürelerinin ortalaması

In [ ]:
# Classes

class Chromosome:
    def __init__(self, chr_id=None):
        self.chr_id = chr_id
        self.ddrule = -1
        self.dsprule = -1
        self.routes = None
        self.env = sim.Environment(trace=False)
        self.env.jobs = sim.Queue('.jobs')
        self.env.joblist = [Job(chromosome=self, job_id=i)
                            for i in range(ps.job_size)]
        self.env.machinelist = [
            Machine(chromosome=self, mc_id=_) for _ in range(ps.machine_size)]
        self.genes = []
        self.fitness = 0

    def generate_genes(self):
        self.reset_genes()
        self.ddrule = np.random.randint(0, ddrule_size)
        self.dsprule = np.random.randint(0, dsprule_size)
        self.routes = np.random.randint(0, ps.route_size, size=ps.job_size)
        self.genes.append(self.ddrule)
        self.genes.append(self.dsprule)
        for r in self.routes:
            self.genes.append(r)

    def generate_data(self):
        self.env.machine_numbers = []
        self.env.operation_durations = []
        for i, j in enumerate(machine_numbers):
            self.env.machine_numbers.append(j[self.routes[i]])
        for i, j in enumerate(operation_durations):
            self.env.operation_durations.append(j[self.routes[i]])

    def reset_genes(self):
        self.ddrule = None
        self.dsprule = None
        self.routes = None
        self.genes = []

    def reset_job_machines(self):
        del self.env.joblist[:]
        del self.env.machinelist[:]

    def change_genes(self, genes):
        self.reset_genes()
        self.ddrule = genes[0]
        self.dsprule = genes[1]
        self.routes = genes[2:]
        self.genes = []
        self.genes.append(self.ddrule)
        self.genes.append(self.dsprule)
        for r in self.routes:
            self.genes.append(r)

    def due_date_assignment(self, q=1.4, k=2.5, w_k=0.6):
#         q=0.4, k=1.5, w_k=0.3 olarak değiştiğinde tardy işler çıktı
        pav = sum(sum(self.env.operation_durations)) / ps.job_size
        if self.ddrule in [0, 3, 19, 22, 25, 28, 31, 34]:
            # print("q değeri için girildi")
            q = q * pav
            # print("q", q)
        elif self.ddrule in [1, 4, 20, 23, 26, 29, 32, 35]:
            q = (q+0.5)*pav
        elif self.ddrule in [2, 5, 21, 24, 27, 30, 33, 36]:
            q = (q+1) * pav
        if self.ddrule in [6, 9, 12, 15, 19, 20, 21, 28, 29, 30]:
            twkk = k
        elif self.ddrule in [7, 10, 13, 16, 22, 23, 24, 31, 32, 33]:
            twkk = k+1
        elif self.ddrule in [8, 11, 14, 17, 25, 26, 27, 34, 35, 36]:
            twkk = k+2
        if 0 <= self.ddrule <= 2:  # SLK Test edildi
            for i, job in enumerate(self.env.joblist):
                """print("iş{} için geliş zamanı {}, proses zamanı: {}, pav değeri: {}".format(
                    i, job.arrival_time, sum(self.env.operation_durations[i]), q))"""
                job.due_date_time = job.arrival_time + \
                    sum(self.env.operation_durations[i]) + q
        elif 3 <= self.ddrule <= 5:  # WSLK # Ağırlık grubu sayısına göre bölündü Test edildi #TODO k parametresi ile optimize edilecek.
            # Ağırlığı küçük olana büyük katsayı, büyük olana küçük katsayı veriyoruz
            for i, job in enumerate(self.env.joblist):
                job.due_date_time = job.arrival_time + \
                    sum(self.env.operation_durations[i]) + w_k*(1/job.weight)*q
        elif 6 <= self.ddrule <= 8:  # TWK Test edildi
            for i, job in enumerate(self.env.joblist):
                job.due_date_time = job.arrival_time+twkk * \
                    sum(self.env.operation_durations[i])
        elif 9 <= self.ddrule <= 11:  # WTWK Job.weight ile mi çarpılacak?
            for i, job in enumerate(self.env.joblist):
                job.due_date_time = job.arrival_time + \
                    w_k*(1/job.weight) * twkk * \
                    sum(self.env.operation_durations[i])
        elif 12 <= self.ddrule <= 14:  # NOPPT
            for i, job in enumerate(self.env.joblist):
                #p1 = 0.5 * twkk
                #p2 = 0.5 * twkk
                # result = 5 * p1 * (ps.job_size * ps.operation_size) + p2 * sum(
                #    self.env.operation_durations[i]) + job.arrival_time  # TODO Noper toplam operasyon sayısı mı?
                # if result <= sum(self.env.operation_durations[i]):
                #    result = sum(self.env.operation_durations[i])
                #job.due_date_time = result
                job.due_date_time = job.arrival_time + \
                    sum(self.env.operation_durations[i]
                        ) + 5*twkk*len(operation_durations)
        elif 15 <= self.ddrule <= 17:  # WNOPPT
            for i, job in enumerate(self.env.joblist):
                job.due_date_time = job.arrival_time+w_k*(1/job.weight) * \
                    sum(self.env.operation_durations[i]) + \
                    5*w_k*(1/job.weight)*twkk*len(operation_durations)
        elif self.ddrule == 18:  # RDM
            """pav ve 3pav  üretilen normal dağılıma uyan bir sayı ile geliş zamanını topladık."""
            for i, job in enumerate(self.env.joblist):
                X = get_truncated_normal(mean=3*pav, sd=pav, low=0, upp=6*pav)
                job.due_date_time = job.arrival_time + X.rvs()
        elif 19 <= self.ddrule <= 27:  # PPW
            for i, job in enumerate(self.env.joblist):
                job.due_date_time = job.arrival_time+q+twkk * \
                    sum(self.env.operation_durations[i])
        elif 28 <= self.ddrule <= 36:  # WPPW
            for i, job in enumerate(self.env.joblist):
                job.due_date_time = job.arrival_time + w_k*(1/job.weight)*0.75*q+w_k*(1/job.weight) * 0.75*twkk * \
                    sum(self.env.operation_durations[i])

    def run_simulation(self, yazdir=False):
        self.fitness = 0.0
        self.env.trace(yazdir)
        self.env.run()

    def calculate_fitness(self):
        performance = 0
        for i, job in enumerate(self.env.joblist):
            tardiness = max(job.departure_time-job.due_date_time, 0)
            earliness = max(job.due_date_time-job.departure_time, 0)
            penalty_dd = job.weight * \
                ((8/480)*(job.due_date_time -job.arrival_time))
            """print("tardiness: {}, earliness: {}, penalty_dd: {}".format(
                tardiness, earliness, penalty_dd))"""
            if tardiness != 0:
                penalty_t = job.weight * (6+6*(tardiness/480))
            else:
                penalty_t = 0
            if earliness != 0:
                penalty_e = job.weight * (5+4*(earliness/480))
            else:
                penalty_e = 0
            penalty_total = penalty_dd+penalty_e+penalty_t
            performance += penalty_total
            """print("penalty_t: {}, penalty_e: {}, penalty_dd: {}, penalty_total: {}, performance: {}".format(
                penalty_t, penalty_e, penalty_dd, penalty_total, performance))"""
        self.fitness = performance

    def __str__(self):
        return self.genes.__str__()

    def __repr__(self):
        return self.genes.__str__()

    def __eq__(self):
        return self.genes.__str__()


class Machine(sim.Component):

    def setup(self, chromosome, mc_id):
        self.mc_id = mc_id
        self.queue = sim.Queue(self.name() + '.queue')
        self.chromosome = chromosome
        self.name('machine' + str(mc_id))
        self.op_list = []

    def process(self):
        while True:
            sel_operation = None
            yield self.hold(0)
            if self.queue:  # there is an operation in queue
                # Dispatching rules
                if self.chromosome.dsprule == 0:  # WATC
                    k = 1
                    # sel_operation = self.queue.head()
                    # print"self.queue: {}".format(self.queue))
                    watc = 0
                    for operation in self.queue:
                        # printoperation)
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        # print"processing_time: {}".format(processing_time))
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        p_ort = sum(
                            op.duration / ps.operation_size for op in operation.job.operation_list)
                        # printp_ort, "p_ort")
                        this_watc = (operation.job.weight / operation.duration) * \
                            (2.71**((max(dif, 0)) / (k * p_ort)))
                        # printthis_watc,"this_watc")
                        if this_watc > watc:
                            watc = this_watc
                            sel_operation = operation
                            # printsel_operation, "sel_operation")
                elif self.chromosome.dsprule == 1:  # WATC
                    k = 2
                    # sel_operation = self.queue.head()
                    watc = 0
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        p_ort = sum(
                            op.duration / ps.operation_size for op in operation.job.operation_list)
                        this_watc = (operation.job.weight / operation.duration) * \
                            (2.71**((max(dif, 0)) / (k * p_ort)))
                        if this_watc > watc:
                            watc = this_watc
                            sel_operation = operation
                elif self.chromosome.dsprule == 2:  # WATC
                    k = 3
                    # sel_operation = self.queue.head()
                    watc = 0
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        p_ort = sum(
                            op.duration / ps.operation_size for op in operation.job.operation_list)
                        this_watc = (operation.job.weight / operation.duration) * \
                            (2.71**((max(dif, 0)) / (k * p_ort)))
                        if this_watc > watc:
                            watc = this_watc
                            sel_operation = operation
                elif self.chromosome.dsprule == 3:  # ATC
                    k = 1
                    # sel_operation = self.queue.head()
                    atc = 0
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        p_ort = sum(
                            op.duration / ps.operation_size for op in operation.job.operation_list)
                        this_atc = (1/operation.duration) * \
                            (2.71**((max(dif, 0)) / (k * p_ort)))
                        if this_atc > atc:
                            atc = this_atc
                            sel_operation = operation
                elif self.chromosome.dsprule == 4:  # ATC
                    k = 2
                    # sel_operation = self.queue.head()
                    atc = 0
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        p_ort = sum(
                            op.duration / ps.operation_size for op in operation.job.operation_list)
                        this_atc = (1/operation.duration) * \
                            (2.71**((max(dif, 0)) / (k * p_ort)))
                        if this_atc > atc:
                            atc = this_atc
                            sel_operation = operation
                elif self.chromosome.dsprule == 5:  # ATC
                    k = 3
                    # sel_operation = self.queue.head()
                    atc = 0
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        p_ort = sum(
                            op.duration / ps.operation_size for op in operation.job.operation_list)
                        this_atc = (1/operation.duration) * \
                            (2.71**((max(dif, 0)) / (k * p_ort)))
                        if this_atc > atc:
                            atc = this_atc
                            sel_operation = operation
                elif self.chromosome.dsprule == 6:  # WMS
                    # sel_operation = self.queue.head()
                    slack = -50000
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        this_slack = -dif*operation.job.weight
                        if this_slack > slack:
                            slack = this_slack
                            sel_operation = operation
                elif self.chromosome.dsprule == 7:  # MS
                    # sel_operation = self.queue.head()
                    slack = -50000
                    for operation in self.queue:
                        processing_time = sum(
                            op.duration for op in operation.job.operation_list)
                        dif = operation.job.due_date_time - processing_time - self.chromosome.env.now()
                        this_slack = -dif
                        if this_slack > slack:
                            slack = this_slack
                            sel_operation = operation
                elif self.chromosome.dsprule == 8:  # WSPT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = sum(
                            op.job.weight/op.duration for op in operation.job.operations)
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 9:  # SPT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = sum(
                            1/op.duration for op in operation.job.operations)
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 10:  # WLPT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = sum(
                            op.duration / op.job.weight for op in operation.job.operations)
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 11:  # LPT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = sum(
                            op.duration for op in operation.job.operations)
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 12:  # WSOT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = operation.job.weight/operation.duration
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 13:  # SOT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = 1/operation.duration
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 14:  # WLOT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = operation.duration/operation.job.weight
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 15:  # LOT
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = operation.duration
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 16:  # EDD
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = 1/operation.job.due_date_time
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 17:  # WEDD
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = operation.job.weight/operation.job.due_date_time
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 18:  # ERD
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = 1/operation.job.arrival_time
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 19:  # WERD
                    # sel_operation = self.queue.head()
                    sel_sum = 0
                    for operation in self.queue:
                        this_sum = operation.job.weight/operation.job.arrival_time
                        if this_sum > sel_sum:
                            sel_sum = this_sum
                            sel_operation = operation
                elif self.chromosome.dsprule == 20:  # SIRO
                    sel_operation = np.random.choice(self.queue)
                elif self.chromosome.dsprule == 21:  # FIFO
                    sel_operation = self.queue.head()
                elif self.chromosome.dsprule == 22:  # LIFO
                    sel_operation = self.queue.tail()
                else:
                    sel_operation = np.random.choice(self.queue)
                sel_operation.start_time = self.chromosome.env.now()
                yield self.hold(sel_operation.duration)
                sel_operation.leave()  # leave both job operations and machine queue
                sel_operation.finish_time = self.chromosome.env.now()  # for departure times
                sel_operation.job.activate()
            else:
                yield self.passivate()


class Operation(sim.Component):
    def setup(self, job, machine, duration):
        self.job = job
        self.machine = machine
        self.duration = duration
        self.start_time = 0
        self.finish_time = 0

    def __str__(self):
        return str(self.job)


class Job(sim.Component):
    def setup(self, job_id, chromosome):
        self.job_id = job_id
        self.name('job' + str(job_id))
        self.chromosome = chromosome
        self.operations = sim.Queue(name=self.name() + '.operations')
        self.operation_list = []
        self.arrival_time = arrival_times[job_id]
        self.weight = weights[job_id]
        self.due_date_time = -1
        self.departure_time = -1
        self.color = None

    def process(self):
        yield self.hold(self.arrival_time)
        self.enter(self.chromosome.env.jobs)
        for j in range(ps.operation_size):
            machine = self.chromosome.env.machinelist[self.chromosome.env.machine_numbers[self.job_id][j]]
            duration = self.chromosome.env.operation_durations[self.job_id][j]
            op = Operation(
                name='operation.' +
                str(self.job_id) + '.' + str(j) + ' m=' +
                machine.name() + ' d=' + str(duration),
                job=self, machine=machine, duration=duration)
            op.enter(self.operations)
            self.operation_list.append(op)
        while self.operations:
            next_operation = self.operations.head()
            next_operation.enter(next_operation.machine.queue)
            if next_operation.machine.ispassive():
                next_operation.machine.activate()
            yield self.passivate()
        self.leave()
        self.departure_time = self.chromosome.env.now()
        if self.chromosome.env.peek() == sim.inf:  # check for end of simulation
            self.chromosome.env.main().activate()

    def __str__(self):
        return 'job'+self.job_id.__str__()

    def __repr__(self):
        return 'job'+self.job_id.__str__()


class Population:

    def __init__(self, size):
        self.size = size
        self.chromosomes = []
        self.best = 0
        self.avg = 0
        self.worst = 0

    def initialize_population(self, gen_num):
        for i in range(self.size):
            chrom = Chromosome(chr_id=i)
            self.chromosomes.append(chrom)
        for ind in self.chromosomes:
            ind.generate_genes()
            ind.generate_data()
            ind.due_date_assignment()
            ind.run_simulation()
            ind.calculate_fitness()
        self.sort_pop()
        self.print_pop(generation_number=gen_num)

    def sort_pop(self):
        return self.chromosomes.sort(key=lambda x: x.fitness, reverse=False)

    def print_pop(self, generation_number):
        # best avg worst
        if mode == 'ga':
            ga_text_file = codecs.open("results\\ga-"+str(at_no)+".txt", "a+")
            self.best = round(self.chromosomes[0].fitness, 3)
            _total = 0
            for ind in self.chromosomes:
                _total += ind.fitness
            self.avg = round(_total / self.size, 2)
            self.worst = round(self.chromosomes[-1].fitness, 2)
            ga_text_file.write(
                "--------------------------------------------------\n")
            ga_text_file.write(
                "Population # {} | Best: {} Avg: {}, Worst: {} ".format(generation_number, self.best, self.avg, self.worst))
            ga_text_file.write(
                "\n--------------------------------------------------\n")
            for i, x in enumerate(self.chromosomes):
                ga_text_file.write("Individual #{} {} |Fitness: {}\n".format(
                    i, x.genes, round(x.fitness, 2)))
            ga_text_file.close()

        elif mode == 'ha':
            ha_text_file = open("ha-"+str(at_no)+".txt", "a+")
            self.best = round(self.chromosomes[0].fitness, 3)
            _total = 0
            for ind in self.chromosomes:
                _total += ind.fitness
            self.avg = round(_total / self.size, 2)
            self.worst = round(self.chromosomes[-1].fitness, 2)
            ha_text_file.write(
                "--------------------------------------------------\n")
            ha_text_file.write(
                "Population # {} | Best: {} Avg: {}, Worst: {} ".format(generation_number, self.best, self.avg, self.worst))
            ha_text_file.write(
                "\n--------------------------------------------------\n")
            for i, x in enumerate(self.chromosomes):
                ha_text_file.write("Individual #{} {} |Fitness: {}\n".format(
                    i, x.genes, round(x.fitness, 2)))
            ha_text_file.close()

    def reap_pop(self):
        self.chromosomes = self.chromosomes[:self.size]

    def __str__(self):
        return self.chromosomes.__str__()

    def __repr__(self):
        return self.chromosomes.__str__()

# Genetik Algoritma
GA rassal arama kullanarak 20 birey oluştur. 20 bireyden en iyi 10 bireyi başlangıç popülasyonu olarak belirle. Başlangıç popülasyonundan ranking seçimine göre 6 adet birey seç ve çaprazlama popülasyonu olarak belirle. Çaprazlama sonucunda 6 adet yeni birey üret ve popülasyona ekle. Başlangıç popülasyonundan 4 adet birey ranking seçimine göre seç ve 4 adet bireye mutasyon uygulayarak yeni 4 adet birey üret. Toplamda oluşan 20 adet bireyi sırala ve en iyi 10 bireyi yeni popülasyon olarak kaydet. Bu döngüye belirlenen iterasyon sayısına kadar devam et.

In [ ]:
# GA/RS
# ESC R Y clear output
np.random.seed(SEED)
start_ga = time.time()
mode = 'ga'

pop_size = 10
ga_iter_size = ps.iter_size
gene_size = ps.job_size + 2
pc = 0.7  # prob cross
pe = 0.5  # prob each genes
pm = 0.3  # prob mutation
num_cross = 6  # number of crossover population
num_mut = 4  # number of mutation population
# number of mutation points in a chromosome
num_points_crossover = ceil(gene_size * 0.1)
num_points_mutation = ceil(gene_size * 0.3)
prob_chr = [0.3, 0.2, 0.15, 0.12, 0.10, 0.07,
            0.03, 0.02, 0.006, 0.004]  # chromosome prob, ranking probability
prob_genes = [0.25, 0.25] + sum_to_x(ps.job_size, 0.5)
# choose genes [0.3, 0.3] + [0.1] toplamları 0.4 olan n tane değer olsun

rs_ga_solutions = []
for i in range(12):
    solution_rs_ga = Chromosome()
    solution_rs_ga.generate_genes()
    solution_rs_ga.generate_data()
    solution_rs_ga.due_date_assignment()
    solution_rs_ga.run_simulation()
    solution_rs_ga.calculate_fitness()
    rs_ga_solutions.append(solution_rs_ga)
#     print(solution_rs_ga.genes)
#     print(solution_rs_ga.fitness)

# Performanslara göre sıraladık
rs_ga_solutions.sort(key=lambda x: x.fitness, reverse=False)
# sıralanmış performanslardan en iyi 10 tanesini kırptık.
rs_ga_solutions_10 = rs_ga_solutions[:10]

# initialize population
pop_list = []
pop = Population(size=pop_size)
for rs_ga in rs_ga_solutions_10:
    pop.chromosomes.append(rs_ga)
#     print(rs_ga.genes,"initial pop")
pop_list.append(pop)

# Genetic Algorithm
for i in range(ga_iter_size):
    new_pop = Population(pop_size)
    new_pop.chromosomes.extend(pop_list[i].chromosomes)
    new_pop_copy = [deepcopy(chr.genes)
                    for chr in new_pop.chromosomes]  # copy genes
    new_pop_copy = np.array(new_pop_copy)
    # crossover
    crossover_pop_idx = np.random.choice(
        range(len(new_pop_copy)), size=num_cross, replace=False, p=prob_chr)  # crossover için genes seç
    crossover_pop_idx.sort()
    #print("iterasyon {}, seçilen bireyler: {}".format(i, crossover_pop_idx))
    crossover_pop = new_pop_copy[crossover_pop_idx, :]
    crossover_pop = np.array(crossover_pop)
    crossover_chromosomes = []
    while len(crossover_chromosomes) < num_cross:
        parents_idx = np.random.choice(
            range(len(crossover_pop)), size=2, replace=False)
        parent1, parent2 = crossover_pop[parents_idx, :]
        parent1 = list(parent1)
        parent2 = list(parent2)
        rs = np.random.random()
        # print("random for crossover: ", rs)
        if rs <= pc:
            if num_points_crossover > 5:
                num_points_crossover = 5
            points = np.random.choice(
                range(gene_size), size=num_points_crossover, replace=False, p=prob_genes)
            points.sort()
            if num_points_crossover == 1:
                child1 = parent1[0:points[0]] + parent2[points[0]:]
                child2 = parent2[0:points[0]] + parent1[points[0]:]
            elif num_points_crossover == 2:
                child1 = parent1[0:points[0]] + \
                    parent2[points[0]:points[1]] + parent1[points[1]:]
                child2 = parent2[0:points[0]] + \
                    parent1[points[0]:points[1]] + parent2[points[1]:]
            elif num_points_crossover == 3:
                child1 = parent1[0:points[0]] + parent2[points[0]:points[1]] + \
                    parent1[points[1]:points[2]] + parent2[points[2]:]
                child2 = parent2[0:points[0]] + parent1[points[0]:points[1]] + \
                    parent2[points[1]:points[2]] + parent1[points[2]:]
            elif num_points_crossover == 4:
                child1 = parent1[0:points[0]] + parent2[points[0]:points[1]] + \
                    parent1[points[1]:points[2]] + \
                    parent2[points[2]:points[3]] + parent1[points[3]:]
                child2 = parent2[0:points[0]] + parent1[points[0]:points[1]] + \
                    parent2[points[1]:points[2]] + \
                    parent1[points[2]:points[3]] + parent2[points[3]:]
            else:
                child1 = parent1[0:points[0]] + parent2[points[0]:points[1]] + \
                    parent1[points[1]:points[2]] + parent2[points[2]:points[3]] + \
                    parent1[points[3]:points[4]] + parent2[points[4]:]
                child2 = parent2[0:points[0]] + parent1[points[0]:points[1]] + \
                    parent2[points[1]:points[2]] + parent1[points[2]:points[3]] + \
                    parent2[points[3]:points[4]] + parent1[points[4]:]
            chr1 = Chromosome()
            chr2 = Chromosome()
            chr1.change_genes(child1)
            chr2.change_genes(child2)
            # print("child1", child1, "popülasyona ekleniyor")
            crossover_chromosomes.append(chr1)
            # print("child2", child2, "popülasyona ekleniyor")
            crossover_chromosomes.append(chr2)
        else:
            chr1 = Chromosome()
            chr2 = Chromosome()
            chr1.change_genes(parent1)
            chr2.change_genes(parent2)
            crossover_chromosomes.append(chr1)
            crossover_chromosomes.append(chr2)
    new_pop.chromosomes.extend(crossover_chromosomes)

    # mutation
    new_pop_copy_mut = [deepcopy(chr.genes) for chr in new_pop.chromosomes]
    new_pop_copy_mut = np.array(new_pop_copy_mut)
    mutation_pop_idx = np.random.choice(
        range(len(new_pop_copy_mut)), size=num_mut)
    mutation_pop_idx.sort()
    mutation_pop = new_pop_copy_mut[mutation_pop_idx, :]
    mutation_pop = np.array(mutation_pop)
    mutation_chromosomes = []

    while len(mutation_chromosomes) < num_mut:
        child_idx = np.random.choice(
            range(len(mutation_pop)), size=1, replace=False)
        child = mutation_pop[child_idx, :]
        rs2 = np.random.random()
        # print("random for mutation: ", rs2)
        if rs2 <= pm:
            mut_points = np.random.choice(
                range(gene_size), size=num_points_mutation, replace=False, p=prob_genes)
            mut_points.sort()
            for mut_point in mut_points:
                # print(mut_point, "mut point")
                if mut_point == 0:  # ddrule
                    child[0][0] = np.random.randint(ddrule_size)
                elif mut_point == 1:  # dsprule
                    child[0][1] = np.random.randint(dsprule_size)
                else:  # routes
                    child[0][mut_point] = np.random.randint(ps.route_size)
            chr3 = Chromosome()
            chr3.change_genes(child[0])
            mutation_chromosomes.append(chr3)
        else:
            chr3 = Chromosome()
            chr3.change_genes(child[0])
            mutation_chromosomes.append(chr3)
    new_pop.chromosomes.extend(mutation_chromosomes)
    for ind in new_pop.chromosomes:
        ind.generate_data()
        ind.due_date_assignment()
        ind.run_simulation()
        ind.calculate_fitness()
    seen_chrs = list()
    new_chromosome_list = []
    for obj in new_pop.chromosomes:
        if obj.genes not in seen_chrs:
            new_chromosome_list.append(obj)
            seen_chrs.append(obj.genes)
    new_pop.chromosomes = new_chromosome_list  # remove chromosome with same genes
    while len(new_pop.chromosomes) < pop_size:
        chr4 = Chromosome()
        random_genes = []
        random_genes.append(np.random.randint(ddrule_size))
        random_genes.append(np.random.randint(dsprule_size))
        random_genes.append(np.random.randint(ps.route_size, size=ps.job_size))
        chr4.change_genes(random_genes)
        new_pop.chromosomes.append(chr4)
    new_pop.sort_pop()  # sort pop
    new_pop.chromosomes = new_pop.chromosomes[:pop_size]  # reap pop
    new_pop.print_pop(generation_number=i + 1)
    pop_list.append(new_pop)

end_ga = time.time()
# loadtxt ile alabilirsin.
np.savetxt(fname='CPU-times\\ga_CPU_times-'+str(at_no) +
           '.txt', X=[end_ga-start_ga], fmt='%1.3f')

ga_perf = []
for populasyon in pop_list:
    ga_perf.append(populasyon.chromosomes[0].fitness)

np.savetxt(fname='best_performances\\ga_perf' +
           str(at_no)+'.txt', X=ga_perf, fmt='%1.3f')

# Komşu arama algoritması (VNS)
Rastgele bir başlangıç çözümü oluştur. Rastgele çözümdeki değişkenlerden birkaçını değiştir ve yeni bir çözüm üret. Eğer çözüm önceki çözümden daha iyiyse çözümü kabul et. Daha kötü bir performansa sahip bir çözüm üretilmesi durumunda üretimi tekrarla.

In [ ]:
# vns
"""Algorithm local search
0. Calculate an initial solution s ~ S;
REPEAT
1. Calculate some solution s' eN(s);
2. If s' is accepted THEN
S := S';
3. UNTIL some stop condition is fulfilled"""
np.random.seed(SEED)
num_points_mutation = ceil(gene_size * 0.3)
gene_size = ps.job_size + 2
start_vns = time.time()
vns_text_file = codecs.open(
    "results\\vns-"+str(at_no)+".txt", "a+", encoding='utf8')


iteration = 0
vns_iter_size = ps.iter_size*10
prob_genes = [0.25, 0.25] + sum_to_x(ps.job_size, 0.5)
solution_x = Chromosome()
solution_x.generate_genes()
solution_x.generate_data()
solution_x.due_date_assignment()
solution_x.run_simulation()
solution_x.calculate_fitness()
vns_text_file.write("initial solution: {} fitness: {} \n".format(
    solution_x, solution_x.fitness))
vns_bests = []
vns_bests.append(solution_x.fitness)
while iteration <= vns_iter_size:
    vns_text_file.write(
        "\n\n----------------iteration:{}----------------\n".format(iteration))
    iteration += 1
    p1_points = np.random.choice(range(
        len(solution_x.genes)), size=num_points_mutation, replace=False, p=prob_genes)
    vns_text_file.write("p1: {}\n".format(p1_points))
    y_genes = deepcopy(solution_x.genes)
    vns_text_file.write("y_genes: {} \n".format(y_genes))
    # Find neighbourhood and change genes

    for p1 in p1_points:
        if p1 == 0:
            while y_genes[int(p1)] == solution_x.genes[int(p1)]:
                y_genes[int(p1)] = np.random.randint(ddrule_size)
        elif p1 == 1:
            while y_genes[int(p1)] == solution_x.genes[int(p1)]:
                y_genes[int(p1)] = np.random.randint(dsprule_size)
        elif p1 in range(2, ps.job_size+2):
            while y_genes[int(p1)] == solution_x.genes[int(p1)]:
                y_genes[int(p1)] = np.random.randint(ps.route_size)

    # Change solution y genes
    solution_y = Chromosome()
    solution_y.change_genes(y_genes)
    solution_y.generate_data()
    solution_y.due_date_assignment()
    solution_y.run_simulation()
    solution_y.calculate_fitness()
    vns_text_file.write("solution y fitness: {} \n".format(solution_y.fitness))
    delta = solution_y.fitness-solution_x.fitness
    vns_text_file.write("delta: {} \n".format(delta))
    if delta < 0:
        vns_text_file.write(
            "solution x y ile değiştirildi. Daha iyi performans")
        solution_x = solution_y
    else:
        vns_text_file.write("x i değiştirmedik")
        solution_x = solution_x
    vns_bests.append(solution_x.fitness)

end_vns = time.time()
np.savetxt(fname='CPU-times\\vns_CPU_times-'+str(at_no)+'.txt',
           X=[end_vns-start_vns], fmt='%1.3f')  # loadtxt ile alabilirsin.
np.savetxt(fname='best_performances\\vns_perf' +
           str(at_no)+'.txt', X=vns_bests, fmt='%1.3f')

# Tavlama Benzetimi (SA)
VNS algoritmasını kullanarak 10 adet birey üret. 10 bireyden en iyisini başlangıç çözümü olarak al. Sıcaklığı 100 olarak belirle. Bir sonraki iterasyonda çözümdeki değişkenlerden birkaçını değiştir. Eğer çözüm daha iyi bir performansa sahipse çözümü kabul et. Eğer daha kötü bir sonuca sahipse Paccept yani kabul olasılığı değerini hesapla. 0-1 arasında rastgele bir sayı üret. Eğer Paccept değeri üretilen rastgele sayıdan büyükse çözümü kabul et. Eğer Paccept değeri rastgele sayıdan küçükse çözümü kabul etme bir önceki çözümle devam et. İterasyon sayısını 1 artır, sıcaklığı belirli oranda düşür ve Paccept olasılığı daha düşük bir olasılığa sahip olsun.

delta/temperature 0.5 ile 5 arasında değişmektedir. exp(-delta/temperature) --> exp(-0.5)=0.6, exp(-5)=0.006 

In [ ]:
# SA
# CPU zamanını ölçmek için
start_sa = time.time()

# sa için yeni bir txt dosyası aç.
sa_text_file = codecs.open(
    "results\\sa-"+str(at_no)+".txt", "a+", encoding='utf8')

iteration = 0
# İterasyon sayısını 10 ile çarp.
sa_iter_size = ps.iter_size*10
# Başlangıç sıcaklığı 50 olarak ayarlandı.
temperature = 50
# gen uzunluğunu belirle
gene_size = ps.job_size + 2
# Kromozomdaki genlerin seçilme olasılıkları
prob_genes = [0.25, 0.25] + sum_to_x(ps.job_size, 0.5)
# Mutasyona uğrayacak gen sayısı ceil: yukarı yuvarlar.
num_points_mutation = ceil(gene_size*0.3)

# RS part
# rassal çözümdeki bireyler için bir liste oluştur.
rs_sa_solutions = []
# 10'da 1'i kadar rastgele çözüm üret.
for i in range(int(ps.iter_size*0.2)):
    # Bir kromozom oluştur.
    solution_rs = Chromosome()
    solution_rs.generate_genes()
    solution_rs.generate_data()
    solution_rs.due_date_assignment()
    solution_rs.run_simulation()
    solution_rs.calculate_fitness()
    # Rastgele çözüm objelerini rs_sa_solutions listesine ekle.
    rs_sa_solutions.append(solution_rs)
    print("{}. rassal çözüm bireyinin uygunluk değeri: {}, bireydeki genler: {}".format(
        i, solution_rs.fitness, solution_rs.genes))

print("iterasyonlar tamamlandı.")

# objelerin uygunluk değerlerini bir listede tut.
tum_performanslar = [
    rs_sa_solutions[i].fitness for i in range(len(rs_sa_solutions))]
print("tum performanslar: {}".format(tum_performanslar))

idx_min = tum_performanslar.index(min(tum_performanslar))
print("idx_min: {}".format(idx_min))

solution_x = rs_sa_solutions[idx_min]
print("başlangıç çözümü uygunluk değeri: {}".format(solution_x.fitness))
sa_perf = []  # tüm iterasyonlardaki üretilen çözümlerin performansları
# şimdiye kadar üretilenlerden daha iyi performansa sahip olan çözümlerin performansları
sa_performances_best = []

while iteration <= sa_iter_size:
    sa_text_file.write(
        "\n\n----------------iteration:{}----------------\n".format(iteration))
    sa_text_file.write("current solution: {} fitness: {} \n".format(
        solution_x, solution_x.fitness))
    print("\n\n----------------iteration:{}----------------\n".format(iteration))
    print("current solution: {} fitness: {} \n".format(solution_x, solution_x.fitness))
    iteration += 1
    p1_points = np.random.choice(range(
        len(solution_x.genes)), size=num_points_mutation, replace=False, p=prob_genes)
    print("mutasyon için seçilen noktalar: {}".format(p1_points))
    sa_text_file.write("değiştirilecek noktalar: {}\n".format(p1_points))
    y_genes = deepcopy(solution_x.genes)
    #sa_text_file.write("x in genleri y ye kopyalandı\n")
    print("x in genlerini y ye kopyaladık")

    # Find neighbourhood and change genes
    for p1 in p1_points:
        print("{} alındı".format(p1))
        if p1 == 0:
            print("p1=0 a girildi")
            #sa_text_file.write("0. gen değiştiriliyor \n")
            while y_genes[int(p1)] == solution_x.genes[int(p1)]:
                y_genes[int(p1)] = np.random.randint(ddrule_size)
            #sa_text_file.write("değişti, yeni genler: {} \n".format(y_genes))
        elif p1 == 1:
            #sa_text_file.write("1. gen değiştiriliyor \n")
            print("p1=1 a girildi")
            while y_genes[int(p1)] == solution_x.genes[int(p1)]:
                y_genes[int(p1)] = np.random.randint(dsprule_size)
            #sa_text_file.write("değişti, yeni genler {}\n".format(y_genes))
        elif p1 in range(2, ps.job_size+2):
            print("p1=2 a girildi")
            #sa_text_file.write("2...n. genler değiştiriliyor \n")
            while y_genes[int(p1)] == solution_x.genes[int(p1)]:
                y_genes[int(p1)] = np.random.randint(ps.route_size)
            #sa_text_file.write("değişti, yeni genler {} \n ".format(y_genes))

    # Change solution y genes
    print("yeni komşu oluşturuldu")
    sa_text_file.write("yeni çözüm değerlendirilmek üzere oluşturuldu\n")
    solution_y = Chromosome()
    solution_y.change_genes(y_genes)
    solution_y.generate_data()
    solution_y.due_date_assignment()
    solution_y.run_simulation()
    solution_y.calculate_fitness()
    print("yeni çözümün genleri: {}, uygunluk değeri: {}".format(solution_y.genes,solution_y.fitness))
    #sa_text_file.write("yeni çözüm değerlendirildi\n")
    #sa_text_file.write("y_genes: {}\n".format(y_genes))
    sa_text_file.write(
        "yeni çözümün performansı: {}\n".format(solution_y.fitness))
    delta = solution_y.fitness-solution_x.fitness
    sa_text_file.write(
        "bir önceki çözümle arasındaki farkı: {}\n".format(delta))
    #print("new solution: {}, fitness: {}".format(solution_y,solution_y.fitness))

    if delta < 0:  # negatif çıkarsa solution y solution x ten daha iyi performansa sahiptir denir.
        print("delta sıfırdan küçük çıktı yani değiştirilmelidir")
        sa_text_file.write(
            "daha iyi performansa sahip olduğu için yeni çözüm kabul edilir.\n")
        #sa_text_file.write("x çözümünün genlerini değiştirdik.\n")
        solution_x = solution_y
        sa_perf.append(solution_x.fitness)  # normal çözüme dahil ettik.
        #print("solution_x: {}, fitness: {}, solution_y: {}, solution_y.fitness: {}".format(solution_x, solution_x.fitness, solution_y, solution_y.fitness))
        #print("sa_performances {}".format(sa_performances))
        #print("sa_performances e dahil edildi\n")
        if min(sa_perf) >= solution_x.fitness:
            #print("en iyi performanslara eklendi")
            sa_performances_best.append(
                solution_y.fitness)  # en iyilere eklendi
            #print("en iyi performanslar: {}".format(sa_performances_best))
    else:
        # delta arttıkça yani aradaki fark büyüdükçe boltz düşüyor. yani seçilme olasılığı düşüyor.
        sa_text_file.write("daha kötü performans ancak p_accepte bakılıyor\n")
        p_accept = min(0.25, (exp(-float(delta*100/solution_y.fitness) /
                                  (temperature-(50/sa_iter_size)))))
        sa_text_file.write("solution_y.fitness: {}, delta: {}, p_accept: {}\n".format(
            solution_y.fitness, delta, p_accept))
        r = np.random.rand()
        #print("kabul oranı: {}, r: {} ".format(p_accept, r))
        sa_text_file.write("r: {}\n".format(r))
        if p_accept >= r:
            sa_text_file.write("kabul oranı r den büyük çıktı.")
            # print("kabul edildi")
            solution_x = solution_y
            sa_text_file.write("x i y ile değiştirdik")
            sa_perf.append(solution_x.fitness)
            if min(sa_perf) >= solution_x.fitness:
                print("en iyi performanslara eklendi")
                sa_performances_best.append(
                    solution_x.fitness)  # en iyilere eklendi
        else:
            sa_text_file.write("x i değiştirmedik")
            sa_perf.append(solution_x.fitness)

end_sa = time.time()
# loadtxt ile alabilirsin.
np.savetxt(fname='CPU-times\\sa_CPU_times-'+str(at_no) +
           '.txt', X=exec_time(start_sa, end_sa), fmt='%1.3f')
np.savetxt(fname='best_performances\\sa_perf' +
           str(at_no)+'.txt', X=sa_perf, fmt='%1.3f')

In [ ]:
# SA2
# CPU zamanını ölçmek için
np.random.seed(SEED)
from random import choice
start_sa = time.time()

# sa için yeni bir txt dosyası aç.
sa_text_file = codecs.open(
    "results\\sa-"+str(at_no)+".txt", "a+", encoding='utf8')

# İterasyon sayısını 10 ile çarp.
sa_iter_size = ps.iter_size*10
# Başlangıç sıcaklığı 100 olarak ayarlandı.
temperature = 100
# gen uzunluğunu belirle
gene_size = ps.job_size + 2
# Kromozomdaki genlerin seçilme olasılıkları
prob_genes = [0.25, 0.25] + sum_to_x(ps.job_size, 0.5)
# Mutasyona uğrayacak gen sayısı ceil: yukarı yuvarlar.
num_points_mutation = ceil(gene_size*0.3)

# RS part
# 10'da 1'i kadar rastgele çözüm üret.
best = 100000 # büyük bir sayı
for i in range(12):
    # Bir kromozom oluştur.
    solution_rs = Chromosome()
    solution_rs.generate_genes()
    solution_rs.generate_data()
    solution_rs.due_date_assignment()
    solution_rs.run_simulation()
    solution_rs.calculate_fitness()
#     print(solution_rs.genes)
#     print(solution_rs.fitness)
    if solution_rs.fitness < best:
        best = solution_rs.fitness
        solution_x = solution_rs

# print("solution x", solution_x.genes, solution_x.fitness)
sa_perf = []  # tüm iterasyonlardaki üretilen çözümlerin performansları
sa_iteration=0
while temperature > 0:  # added
    sa_iteration+=1
    sa_text_file.write(
        "\n\n----------------temperature:{}----------------\n".format(temperature))
    sa_text_file.write("current solution: {} fitness: {} \n".format(
        solution_x, solution_x.fitness))
    # solution_y
    p1_points = np.random.choice(range(
        len(solution_x.genes)), size=num_points_mutation, replace=False, p=prob_genes)
#     print(p1_points)
    y_genes = deepcopy(solution_x.genes)
    sa_text_file.write("x in genleri y ye kopyalandı\n")

    # Find neighbourhood and change genes
    for p1 in p1_points:
        if p1 == 0:
            y_genes[0]=choice([x for x in range(ddrule_size) if x != y_genes[0]])
#             while y_genes[int(p1)] == solution_x.genes[int(p1)]:
#                 y_genes[int(p1)] = np.random.randint(ddrule_size)
        elif p1 == 1:
            y_genes[1]=choice([x for x in range(dsprule_size) if x != y_genes[1]])
        elif p1 in range(2, ps.job_size+2):
            y_genes[int(p1)]=choice([x for x in range(ps.route_size) if x != y_genes[int(p1)]])
#             while y_genes[int(p1)] == solution_x.genes[int(p1)]:
#                 y_genes[int(p1)] = np.random.randint(ps.route_size)

    # Change solution y genes
    sa_text_file.write("yeni çözüm değerlendirilmek üzere oluşturuldu\n")
    solution_y = Chromosome()
    solution_y.change_genes(y_genes)
    solution_y.generate_data()
    solution_y.due_date_assignment()
    solution_y.run_simulation()
    solution_y.calculate_fitness()
#     print(solution_y.genes,"solution y genes")
    sa_text_file.write(
        "yeni çözümün performansı: {}\n".format(solution_y.fitness))
    delta = solution_y.fitness-solution_x.fitness
    sa_text_file.write(
        "bir önceki çözümle arasındaki farkı: {}\n".format(delta))

    if delta < 0:
        sa_text_file.write(
            "daha iyi performansa sahip olduğu için yeni çözüm kabul edilir.\n")
        solution_x = solution_y
        
    else:
        increase_rate = delta/solution_x.fitness
#         print(increase_rate, "increase rate")
        if increase_rate == 0:
            p_accept = 0.001
        elif 0<increase_rate<1:
            p_accept = min(0.4,exp(-(increase_rate*3000) / temperature))
        elif increase_rate >=1:
            p_accept = 0.001
        sa_text_file.write("daha kötü performans ancak p_accepte bakılıyor\n")
#         print(p_accept)
        sa_text_file.write("solution_y.fitness: {}, delta: {}, p_accept: {}\n".format(
            solution_y.fitness, delta, p_accept))
        r = np.random.random()
        if r < p_accept:
            sa_text_file.write("x i y ile değiştirdik")
            solution_x = solution_y
        else:
            sa_text_file.write("x i değiştirmedik")
    sa_perf.append(solution_x.fitness)
    temperature -= 100/sa_iter_size  # added

end_sa = time.time()
# loadtxt ile alabilirsin.
np.savetxt(fname='CPU-times\\sa_CPU_times-'+str(at_no) +
           '.txt', X=[end_sa-start_sa], fmt='%1.3f')
np.savetxt(fname='best_performances\\sa_perf' +
           str(at_no)+'.txt', X=sa_perf, fmt='%1.3f')

# Tabu Arama (TA)
VNS algoritmasını kullanarak 10 adet birey üret. 10 bireyden en iyisini başlangıç çözümü olarak al. Tabu listesini oluştur. Bir iterasyonda yeni bir çözüm üret. Eğer üretilen çözüm önceki çözümden daha iyi bir performansa sahipse çözümü kabul et. Eğer daha kötü bir performansa sahipse çözümü kabul etme ve üretilen çözümü tabu listesine koy. Bir sonraki iterasyonda üretilen çözümlerin tabu listesine olmamasını sağla. Her iterasyonda tabu listesini güncelle.

In [ ]:
# TA
"""Algorithm tabu search
T:=0;
0. Calculate an initial solution s ~ S;
REPEAT
IF all modifications lead to solutions which are tabu
THEN
STOP;
1. Choose an allowed modification F which does not
lead to a tabu solution;
Calculate the resulting solution s' := F(s);
2. s := s';
update the tabu list;
3. UNTIL some stop-condition is fulfilled; """
np.random.seed(SEED)
start_ta = time.time()
ta_text_file = codecs.open(
    "results\\ta-"+str(at_no)+".txt", "a+", encoding='utf8')


iteration = 0
ta_iter_size = ps.iter_size*10
tabu_list = []
ta_perf = []
gene_size = ps.job_size+2
num_points_mutation = ceil(gene_size * 0.3)

# RS part
rs_ta_solutions = []
for i in range(12):
    solution_rs_ta = Chromosome()
    solution_rs_ta.generate_genes()
    solution_rs_ta.generate_data()
    solution_rs_ta.due_date_assignment()
    solution_rs_ta.run_simulation()
    solution_rs_ta.calculate_fitness()
    rs_ta_solutions.append(solution_rs_ta)

tum_performanslar_ta = [rs_ta_solutions[i].fitness for i in range(len(rs_ta_solutions))]
print(tum_performanslar_ta)
idx_min_ta = tum_performanslar_ta.index(min(tum_performanslar_ta))

tabu_solution_x = rs_ta_solutions[idx_min_ta]
#print("başlangıç çözümü: {}".format(tabu_solution_x))

ta_text_file.write("tabu_solution_x: {} \n".format(tabu_solution_x))

while iteration <= ta_iter_size:
    ta_text_file.write(
        "\n\n----------------iteration:{}----------------\n".format(iteration))
    tabu_y_genes = deepcopy(tabu_solution_x.genes)
    ta_text_file.write("tabu_x_genes: {} \n".format(tabu_y_genes))
    #print("iterasyon {}".format(iteration))
    # Find neighbourhood and change genes
    p1_points_ta = np.random.choice(
        range(len(tabu_solution_x.genes)), size=num_points_mutation, replace=False)
    #print("seçilen noktalar, {}".format(p1_points_ta))
    #print("tabu_listesi, {}".format(tabu_list))
    ta_text_file.write("p1: {} \n".format(p1_points_ta))

    for p1 in p1_points_ta:
        if p1 == 0:
            #ta_text_file.write("p1==0 a girildi")
            while tabu_y_genes[int(p1)] == tabu_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(ddrule_size)
            #gata_text_file.write("değişen: {} \n".format(tabu_y_genes))
        elif p1 == 1:
            #gata_text_file.write("p1==1 e girildi")
            while tabu_y_genes[int(p1)] == tabu_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(dsprule_size)
            #gata_text_file.write("değişen: {} \n".format(tabu_y_genes))
        elif p1 in range(2, ps.job_size+2):
            #gata_text_file.write("p1==2: e girildi")
            while tabu_y_genes[int(p1)] == tabu_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(ps.route_size)
            #gata_text_file.write("değişen: {} \n".format(tabu_y_genes))

    while tabu_y_genes in tabu_list:
        p1_points_ta = np.random.choice(
            range(len(tabu_solution_x.genes)), size=num_points_mutation, replace=False)
        #print("while döngüsüne girildi")
        for p1 in p1_points_ta:
            #print("p1: {}".format(p1))
            if p1 == 0:
                #ta_text_file.write("p1==0 a girildi")
                #print("p1==0 a girildi")
                while tabu_y_genes[int(p1)] == tabu_solution_x.genes[int(p1)]:
                    tabu_y_genes[int(p1)] = np.random.randint(ddrule_size)
                #ta_text_file.write("değişen: {} \n".format(tabu_y_genes))
            elif p1 == 1:
                #print("p1==1e girildi")
                #ta_text_file.write("p1==1 e girildi")
                while tabu_y_genes[int(p1)] == tabu_solution_x.genes[int(p1)]:
                    tabu_y_genes[int(p1)] = np.random.randint(dsprule_size)
                #ta_text_file.write("değişen: {} \n".format(tabu_y_genes))
            elif p1 in range(2, ps.job_size+2):
                #print("p1==2e girildi")
                #ta_text_file.write("p1==2: e girildi")
                while tabu_y_genes[int(p1)] == tabu_solution_x.genes[int(p1)]:
                    tabu_y_genes[int(p1)] = np.random.randint(ps.route_size)
                #ta_text_file.write("değişen: {} \n".format(tabu_y_genes))

    # Change solution y genes,
    # print(y_genes)
    tabu_solution_y = Chromosome()
    tabu_solution_y.change_genes(tabu_y_genes)
    tabu_solution_y.generate_data()
    tabu_solution_y.due_date_assignment()
    tabu_solution_y.run_simulation()
    tabu_solution_y.calculate_fitness()
    ta_text_file.write(
        "solution y fitness: {} \n".format(tabu_solution_y.fitness))
    #print("tabu_x: {}, tabu_y: {}".format(tabu_solution_x, tabu_solution_y))
    delta = tabu_solution_y.fitness-tabu_solution_x.fitness
    # print(delta)
    ta_text_file.write("delta: {} \n".format(delta))
    if delta <= 0:
        #print("delta sıfırdan küçük çıktı çözümler değiştirildi")
        ta_text_file.write(
            "solution x y ile değiştirildi. Daha iyi performans")
        tabu_solution_x = tabu_solution_y
        #print("tabu_x: {}, tabu_y: {}".format(tabu_solution_x, tabu_solution_y))
    else:
        tabu_list.append(tabu_solution_y.genes)
        tabu_solution_x = tabu_solution_x
        ta_text_file.write("tabu_list: {} \n".format(tabu_list))
    ta_perf.append(tabu_solution_x.fitness)
    iteration += 1

    if (len(tabu_list) > 10):
        ##print("tabu listesi uzunluğu 10'dan fazla")
        tabu_list = tabu_list[1:]
        ##print("son 10 tane tabu alındı")
        ##print(tabu_list, "yeni tabu list")

end_ta = time.time()
# loadtxt ile alabilirsin.
np.savetxt(fname='CPU-times\\ta_CPU_times-'+str(at_no) +
           '.txt', X=[end_ta-start_ta], fmt='%1.3f')
np.savetxt(fname='best_performances\\ta_perf' +
           str(at_no)+'.txt', X=ta_perf, fmt='%1.3f')

# Genetik + Tavlama Algoritması (GA/SA)
İlk önce GA kullanarak genel çözüm arasından iyi sonuçlara ulaşmaya çalış. GA/SA'daki GA'yı GA'daki iterasyon sayısının yarısı kadar çalıştır. Yarısından elde edilen en son popülasyondaki en iyi sonucu SA'nın başlangıç çözümü olarak SA'ya ver. SA adımlarını SA'nın yarısı kadar iterasyonda uygulayarak GA/SA algoritmasını çalıştır.

In [ ]:
# GA/SA
from math import exp, fabs
"""Bir popülasyon oluştur. GA algoritması uygula. Sonra en iyi kromozomu alıp SA algoritmasını uygula."""
np.random.seed(SEED)
start_gasa = time.time()
gasa_text_file = codecs.open(
    "results\\gasa-"+str(at_no)+".txt", "a+", encoding='utf8')
mode = 'ha'

# GA part
iteration = 0
gasa_iter_size = int((ps.iter_size/2)*10)
temperature = 100

gasa_solution_x = pop_list[int(ps.iter_size/2)].chromosomes[0]
# print("initial solution:", gasa_gasa_solution_x, "fitness: ", gasa_solution_x.fitness)
gasa_perf = []
gasa_perf = ga_perf[1:int(ps.iter_size/2)+1]
gasa_iteration=0
while temperature > 0:
    gasa_iteration+=1
    gasa_text_file.write("current solution: {} fitness: {} \n".format(
        gasa_solution_x, gasa_solution_x.fitness))
    p1_points = np.random.choice(range(
        len(gasa_solution_x.genes)), size=num_points_mutation, replace=False, p=prob_genes)
    gasa_text_file.write("değiştirilecek noktalar: {}\n".format(p1_points))
    y_genes = deepcopy(gasa_solution_x.genes)
    gasa_text_file.write("x in genleri y ye kopyalandı\n")
    # print(gasa_perf)

    # Find neighbourhood and change genes
    for p1 in p1_points:
        if p1 == 0:
            y_genes[0]=choice([x for x in range(ddrule_size) if x != y_genes[0]])
#             while y_genes[int(p1)] == solution_x.genes[int(p1)]:
#                 y_genes[int(p1)] = np.random.randint(ddrule_size)
        elif p1 == 1:
            y_genes[1]=choice([x for x in range(dsprule_size) if x != y_genes[1]])
        elif p1 in range(2, ps.job_size+2):
            y_genes[int(p1)]=choice([x for x in range(ps.route_size) if x != y_genes[int(p1)]])

    # Change solution y genes
    gasa_text_file.write("yeni çözüm değerlendirilmek üzere oluşturuldu\n")
    solution_y = Chromosome()
    solution_y.change_genes(y_genes)
    solution_y.generate_data()
    solution_y.due_date_assignment()
    solution_y.run_simulation()
    solution_y.calculate_fitness()
    gasa_text_file.write("yeni çözüm değerlendirildi\n")
    gasa_text_file.write("y_genes: {}\n".format(y_genes))
    gasa_text_file.write(
        "yeni çözümün performansı: {}\n".format(solution_y.fitness))
    delta = solution_y.fitness-gasa_solution_x.fitness
    gasa_text_file.write(
        "bir önceki çözümle arasındaki farkı: {}\n".format(delta))
    #print("new solution: {}, fitness: {}".format(solution_y,solution_y.fitness))

    if delta <= 0:  # negatif çıkarsa solution y solution x ten daha iyi performansa sahiptir denir.
        gasa_text_file.write(
            "daha iyi performansa sahip olduğu için yeni çözüm kabul edilir.\n")
        gasa_text_file.write("x çözümünün genlerini değiştirdik.\n")
        gasa_solution_x = solution_y

    else:
        # delta arttıkça yani aradaki fark büyüdükçe boltz düşüyor. yani seçilme olasılığı düşüyor.
        increase_rate = delta/gasa_solution_x.fitness
#         print(increase_rate, "increase rate")
        if increase_rate == 0:
            p_accept = 0.001
        elif 0<increase_rate<1:
            p_accept = min(0.4,exp(-(increase_rate*3000) / temperature))
        elif increase_rate >=1:
            p_accept = 0.001
        gasa_text_file.write("daha kötü performans ancak p_accepte bakılıyor\n")
#         print(p_accept)
        gasa_text_file.write("solution_y.fitness: {}, delta: {}, p_accept: {}\n".format(
            solution_y.fitness, delta, p_accept))
        r = np.random.random()
        if r < p_accept:
            gasa_text_file.write("x i y ile değiştirdik")
            gasa_solution_x = solution_y
        else:
            gasa_text_file.write("x i değiştirmedik")
        r = np.random.random()
        #print("kabul oranı: {}, r: {} ".format(p_accept, r))
    gasa_perf.append(gasa_solution_x.fitness)
    temperature-=100/gasa_iter_size

end_gasa = time.time()
np.savetxt(fname='CPU-times\\gasa_CPU_times-'+str(at_no)+'.txt',
           X=[(end_ga-start_ga)/2 + (end_gasa-start_gasa)], fmt='%1.3f')  # loadtxt ile alabilirsin.
np.savetxt(fname='best_performances\\gasa_perf' +
           str(at_no)+'.txt', X=gasa_perf, fmt='%1.3f')

# Genetik + Tabu Arama Algoritması (GA/TA)
İlk önce GA kullanarak genel çözüm arasından iyi sonuçlara ulaşmaya çalış. GA/TA'daki GA'yı GA'daki iterasyon sayısının yarısı kadar çalıştır. Yarısından elde edilen en son popülasyondaki en iyi sonucu TA'nın başlangıç çözümü olarak TA'ya ver. TA adımlarını TA'nın yarısı kadar iterasyonda uygulayarak GA/TA algoritmasını çalıştır.

In [ ]:
# GA/TA
"""Bir popülasyon oluştur. GA algoritması uygula. Sonra en iyi kromozomu alıp TA algoritmasını uygula."""
np.random.seed(SEED)
start_gata = time.time()
gata_text_file = codecs.open(
    "results\\gata-"+str(at_no)+".txt", "a+", encoding='utf8')

iteration = 0
gata_iter_size = int((ps.iter_size/2)*10)
gata_solution_x = pop_list[int(ps.iter_size/1.2)].chromosomes[0]
gata_tabu_list = []
#print("initial solution:", solution_x, "fitness: ", solution_x.fitness)
gata_perf = []
gata_perf = ga_perf[:int(ps.iter_size/1.2)]

while iteration <= gata_iter_size:
    gata_text_file.write(
        "\n\n----------------iteration:{}----------------\n".format(iteration))
    tabu_y_genes = deepcopy(gata_solution_x.genes)
    gata_text_file.write("tabu_x_genes: {} \n".format(tabu_y_genes))

    # Find neighbourhood and change genes
    p1_points_ta = np.random.choice(
        range(len(gata_solution_x.genes)), size=num_points_mutation, replace=False)
    gata_text_file.write("p1: {} \n".format(p1_points_ta))
    for p1 in p1_points_ta:
        if p1 == 0:
            gata_text_file.write("p1==0 a girildi")
            while tabu_y_genes[int(p1)] == gata_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(ddrule_size)
            gata_text_file.write("değişen: {} \n".format(tabu_y_genes))
        elif p1 == 1:
            gata_text_file.write("p1==1 e girildi")
            while tabu_y_genes[int(p1)] == gata_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(dsprule_size)
            gata_text_file.write("değişen: {} \n".format(tabu_y_genes))
        elif p1 in range(2, ps.job_size+2):
            gata_text_file.write("p1==2: e girildi")
            while tabu_y_genes[int(p1)] == gata_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(ps.route_size)
            gata_text_file.write("değişen: {} \n".format(tabu_y_genes))

    #print("tabu listesi uzunluğu: ", len(gata_tabu_list))

    if (len(gata_tabu_list) > 10):
        #print("tabu listesi uzunluğu 10'dan fazla")
        gata_tabu_list = gata_tabu_list[1:]
        #print("son 10 tane tabu alındı")
        #print(gata_tabu_list, "yeni tabu list")

    while tabu_y_genes in gata_tabu_list:
        p1 = np.random.choice(
            range(len(gata_solution_x.genes)), size=1, replace=False)
        gata_text_file.write("p1: {} \n".format(p1))
        if p1 == 0:
            gata_text_file.write("p1==0 a girildi")
            while tabu_y_genes[int(p1)] == gata_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(ddrule_size)
            gata_text_file.write("değişen: {} \n".format(tabu_y_genes))
        elif p1 == 1:
            gata_text_file.write("p1==1 e girildi")
            while tabu_y_genes[int(p1)] == gata_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(dsprule_size)
            gata_text_file.write("değişen: {} \n".format(tabu_y_genes))
        elif p1 in range(2, ps.job_size+2):
            gata_text_file.write("p1==2: e girildi")
            while tabu_y_genes[int(p1)] == gata_solution_x.genes[int(p1)]:
                tabu_y_genes[int(p1)] = np.random.randint(ps.route_size)
            gata_text_file.write("değişen: {} \n".format(tabu_y_genes))

    # Change solution y genes
    tabu_solution_y = Chromosome()
    tabu_solution_y.change_genes(tabu_y_genes)
    tabu_solution_y.generate_data()
    tabu_solution_y.due_date_assignment()
    tabu_solution_y.run_simulation()
    tabu_solution_y.calculate_fitness()
    gata_text_file.write(
        "solution y fitness: {} \n".format(tabu_solution_y.fitness))
    delta = tabu_solution_y.fitness-gata_solution_x.fitness
    gata_text_file.write("delta: {} \n".format(delta))
    if delta <= 0:
        gata_text_file.write(
            "solution x y ile değiştirildi. Daha iyi performans")
        gata_solution_x = tabu_solution_y
    else:
        gata_tabu_list.append(tabu_solution_y.genes)
        gata_solution_x = gata_solution_x
        gata_text_file.write("gata_tabu_list: {} \n".format(gata_tabu_list))
    gata_perf.append(gata_solution_x.fitness)
    iteration += 1

end_gata = time.time()
np.savetxt(fname='CPU-times\\gata_CPU_times-'+str(at_no)+'.txt',
           X=[(end_ga-start_ga)/2 + (end_gata-start_gata)], fmt='%1.3f')  # loadtxt ile alabilirsin.
np.savetxt(fname='best_performances\\gata_perf' +
           str(at_no)+'.txt', X=gata_perf, fmt='%1.3f')

# ICPSO

In [ ]:
# ICPSO
icpso_start_time = time.time()
swarm_list = []

gbest = float('inf')
gbest_list=[]
gbest_particle_list=[]
gbest_particle = None


# başlangıç sürüsü
print("-" * 100)
print("Initial Swarm", "w=", Swarm.w, "c1=", Swarm.c_1, "c2=", Swarm.c_2)
print("-" * 100)
print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6} {6:<10}".format("id", "dd", "dsp", "routes",
                                                                  "fitness", "pbest", "swarm: 0"))
print("-" * 100)
initial_swarm = Swarm(size=pop_size,generation=0)
for i in range(pop_size):
    initial_swarm.particles.append(Chromosome(particle_id=i))
for particle in initial_swarm.particles:
    # Randomly generate first swarm
    particle.ddrule_probs = sum_to_x(ddrule_size, sum_to)  # 0 ile 1 arasında ddrule size kadar rastgele.
    # particle.ddrule_probs = [(1 / ddrule_size) for i in range(ddrule_size)]
    particle.dsprule_probs = sum_to_x(dsprule_size, sum_to)
    # particle.dsprule_probs = [(1 / dsprule_size) for i in range(dsprule_size)]
    particle.routes_probs = []
    for i in range(ps.job_size):
        particle.routes_probs.append(sum_to_x(ps.route_size, sum_to))

    # Make sample -- Olasılıklardan değer seç
    particle.ddrule = np.random.choice(Chromosome.ddrule_values, 1, p=particle.ddrule_probs)[0]
    particle.dsprule = np.random.choice(Chromosome.dsprule_values, 1, p=particle.dsprule_probs)[0]
    particle.routes = []
    for i in range(ps.job_size):
        prob = particle.routes_probs[i]
        value = Chromosome.routes_values[i]
        route_value = np.random.choice(value, size=1, p=prob)[0]
        particle.routes.append(route_value)

    # Fitness değerlerini hesapla
    particle.set_positions()
    particle.change_genes(particle.position_values)
    particle.generate_data()
    particle.due_date_assignment()
    particle.run_simulation(yazdir=False)
    particle.calculate_fitness()
    particle.pbest = particle.fitness
    particle.pbest_ddrule_probs = particle.ddrule_probs
    particle.pbest_dsprule_probs = particle.dsprule_probs
    particle.pbest_routes_probs = particle.routes_probs
    particle.pbest_ddrule_value = particle.ddrule
    particle.pbest_dsprule_value = particle.dsprule
    particle.pbest_routes_values = particle.routes
    particle.ddrule_velocity = [0] * ddrule_size
    particle.dsprule_velocity = [0] * dsprule_size
    particle.routes_velocity = []
    for i in range(ps.job_size):
        particle.routes_velocity.append([0] * ps.route_size)

    # eğer sürüdeki bir parçacık gbest'ten daha iyiyse gbest i değiştir.
    if particle.fitness <= gbest:
        gbest = particle.fitness
        gbest_particle = particle
    print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6}".format(particle.particle_id, particle.ddrule,
                                                                      particle.dsprule, str(particle.routes),
                                                                      round(particle.fitness, 2),
                                                                      round(particle.pbest, 2)))

gbest_list.append(gbest) # gbesti listeye ekle
gbest_particle_list.append(gbest_particle)

swarm_list.append(initial_swarm) # başlangıç sürüsünü swarm liste kaydet


# iterasyonlara başla
for i in range(1,ps.iter_size):
    s = Swarm(size=pop_size, generation=i)
    print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6} swarm: {6:<4}".format("id", "dd", "dsp",
                                                                                    "routes",
                                                                                    "fitness", "pbest",
                                                                                    str(s.generation)))
    print("-" * 100)
    
    for p, particle in enumerate(swarm_list[i-1].particles):
        new_particle=Chromosome()
        new_particle.particle_id=p
        
        r1 = np.random.random()
        r2 = np.random.random()
        
        dd_velocity = particle.ddrule_velocity[:]
        dsp_velocity = particle.dsprule_velocity[:]
        route_velocity = particle.routes_velocity[:]
        
        # new ddrule probs
        inertia_ddrule = [x * Swarm.w for x in dd_velocity]
        cognitive_ddrule = [r1 * Swarm.c_1 * (a + b) for a, b in
                            zip(particle.pbest_ddrule_probs, particle.ddrule_probs)]  # TODO
        social_ddrule = [r2 * Swarm.c_2 * (a + b) for a, b in
                         zip(gbest_particle.ddrule_probs, particle.ddrule_probs)]
        new_particle.ddrule_velocity = [a + b + c for a, b, c in
                                        zip(inertia_ddrule, cognitive_ddrule, social_ddrule)]
        new_ddrule_probs = [a + b for a, b in zip(particle.ddrule_probs, new_particle.ddrule_velocity)]
        # Toplamları 1 olacak şekilde oranladık.
        new_particle.ddrule_probs = [x / sum(new_ddrule_probs) for x in new_ddrule_probs]
        # new dsprule
        inertia_dsprule = [x * Swarm.w for x in dsp_velocity]
        cognitive_dsprule = [r1 * Swarm.c_1 * (a + b) for a, b in
                             zip(particle.pbest_dsprule_probs, particle.dsprule_probs)]  # TODO
        social_dsprule = [r2 * Swarm.c_2 * (a + b) for a, b in
                          zip(gbest_particle.dsprule_probs, particle.dsprule_probs)]
        new_particle.dsprule_velocity = [a + b + c for a, b, c in
                                         zip(inertia_dsprule, cognitive_dsprule, social_dsprule)]
        new_dsprule_probs = [a + b for a, b in zip(particle.dsprule_probs, new_particle.dsprule_velocity)]
        new_particle.dsprule_probs = [x / sum(new_dsprule_probs) for x in new_dsprule_probs]

        # new routes
        new_particle.routes_probs = []
        new_particle.routes_velocity = []
        for i in range(ps.job_size):
            # New routes
            inertia_routes = [x * Swarm.w for x in route_velocity[i]]
            cognitive_routes = [r1 * Swarm.c_1 * (a + b) for a, b in
                                zip(particle.pbest_routes_probs[i], particle.routes_probs[i])]  # TODO
            social_routes = [r2 * Swarm.c_2 * (a + b) for a, b in
                             zip(gbest_particle.routes_probs[i], particle.routes_probs[i])]
            rv = [a + b + c for a, b, c in
                  zip(inertia_routes, cognitive_routes, social_routes)]
            new_particle.routes_velocity.append(rv)
            new_routes_probs = [a + b for a, b in
                                zip(particle.routes_probs[i], new_particle.routes_velocity[i])]
            normalized_routes_probs = [x / sum(new_routes_probs) for x in new_routes_probs]
            new_particle.routes_probs.append(normalized_routes_probs)

        # New ddrule value
        new_particle.ddrule = np.random.choice(Chromosome.ddrule_values, size=1, p=new_particle.ddrule_probs)[0]
        # New dsprule value
        new_particle.dsprule = \
            np.random.choice(Chromosome.dsprule_values, size=1, p=new_particle.dsprule_probs)[
                0]
        # New routes value
        new_particle.routes = []
        for i in range(ps.job_size):
            route = np.random.choice(Chromosome.routes_values[i], size=1, p=new_particle.routes_probs[i])[0]
            new_particle.routes.append(route)

        # Calculate new fitness
        new_particle.generate_data()
        new_particle.due_date_assignment()
        new_particle.run_simulation()
        new_particle.calculate_fitness()
        # print(p, ". ", "particle.fitness", particle.fitness)

        # Pbest
        if new_particle.fitness <= particle.pbest:
            new_particle.pbest = new_particle.fitness
            new_particle.pbest_ddrule_value = new_particle.ddrule
            new_particle.pbest_dsprule_value = new_particle.dsprule
            new_particle.pbest_routes_value = new_particle.routes[:]
            # probs
            new_particle.pbest_ddrule_probs = new_particle.ddrule_probs[:]
            new_particle.pbest_dsprule_probs = new_particle.dsprule_probs[:]
            new_particle.pbest_routes_probs = new_particle.routes_probs[:]
        else:
            new_particle.pbest = particle.pbest
            new_particle.pbest_ddrule_value = particle.ddrule
            new_particle.pbest_dsprule_value = particle.dsprule
            new_particle.pbest_routes_value = particle.routes[:]
            # probs
            new_particle.pbest_ddrule_probs = particle.ddrule_probs[:]
            new_particle.pbest_dsprule_probs = particle.dsprule_probs[:]
            new_particle.pbest_routes_probs = particle.routes_probs[:]

        # Gbest
        if new_particle.fitness <= gbest:
            gbest = new_particle.fitness
            gbest_particle = new_particle
        print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6}".format(particle.particle_id, particle.ddrule,
                                                                          particle.dsprule, str(particle.routes),
                                                                          round(particle.fitness, 2),
                                                                          round(particle.pbest, 2)))
        s.particles.append(new_particle)
    swarm_list.append(s)
    print("-" * 100)
    print(
        "gbest: {} gbest_particle: {}\n\n".format(round(gbest, 2), gbest_particle.particle_id))
    gbest_list.append(gbest)
    gbest_particle_list.append(gbest_particle)

# çalıştırma zamanını kaydet
icpso_finish_time = time.time()

np.savetxt(fname='best_performances\\icpso_perf' +
           str(at_no)+'.txt', X=gbest_list, fmt='%1.3f')
np.savetxt(fname='best_performances\\icpso_CPU' +
           str(at_no)+'.txt', X=exec_time(icpso_start_time, icpso_finish_time))
icpso_genes=[gbest_particle.ddrule, gbest_particle.dsprule]+gbest_particle.routes
np.savetxt(fname='best_performances\\icpso_solution' +
           str(at_no)+'.txt', X=icpso_genes)

# MICPSO

In [ ]:
# MICPSO

# Magics
%precision 2

micpso_start_time = time.time()
micpso_swarm_list = []

micpso_gbest = float('inf')
micpso_gbest_list = []
micpso_gbest_particle_list = []
micpso_gbest_particle = None

# başlangıç sürüsü
print("-" * 100)
print("Initial Swarm", "w=", Swarm.w, "c1=", Swarm.c_1, "c2=", Swarm.c_2)
print("-" * 100)
print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6} {6:<10}".format("id", "dd", "dsp", "routes",
                                                                  "fitness", "pbest", "swarm: 0"))
print("-" * 100)
micpso_initial_swarm = Swarm(size=pop_size, generation=0)
for i in range(pop_size):
    micpso_initial_swarm.particles.append(Chromosome(particle_id=i))
for particle in micpso_initial_swarm.particles:
    # Randomly generate first swarm
    # 0 ile 1 arasında ddrule size kadar rastgele.
    particle.ddrule_probs = sum_to_x(ddrule_size, sum_to)
    # particle.ddrule_probs = [(1 / ddrule_size) for i in range(ddrule_size)]
    particle.dsprule_probs = sum_to_x(dsprule_size, sum_to)
    # particle.dsprule_probs = [(1 / dsprule_size) for i in range(dsprule_size)]
    particle.routes_probs = []
    for i in range(ps.job_size):
        particle.routes_probs.append(sum_to_x(ps.route_size, sum_to))

    # Make sample -- Olasılıklardan değer seç
    particle.ddrule = np.random.choice(
        Chromosome.ddrule_values, 1, p=particle.ddrule_probs)[0]
    particle.dsprule = np.random.choice(
        Chromosome.dsprule_values, 1, p=particle.dsprule_probs)[0]
    particle.routes = []
    for i in range(ps.job_size):
        prob = particle.routes_probs[i]
        value = Chromosome.routes_values[i]
        route_value = np.random.choice(value, size=1, p=prob)[0]
        particle.routes.append(route_value)

    # Fitness değerlerini hesapla
    particle.set_positions()
    particle.change_genes(particle.position_values)
    particle.generate_data()
    particle.due_date_assignment()
    particle.run_simulation(yazdir=False)
    particle.calculate_fitness()
    particle.pbest = particle.fitness
    particle.pbest_ddrule_probs = particle.ddrule_probs[:]
    particle.pbest_dsprule_probs = particle.dsprule_probs[:]
    particle.pbest_routes_probs = particle.routes_probs[:]
    particle.pbest_ddrule_value = particle.ddrule
    particle.pbest_dsprule_value = particle.dsprule
    particle.pbest_routes_values = particle.routes
    particle.ddrule_velocity = [0] * ddrule_size
    particle.dsprule_velocity = [0] * dsprule_size
    particle.routes_velocity = []
    for i in range(ps.job_size):
        particle.routes_velocity.append([0] * ps.route_size)

    # eğer sürüdeki bir parçacık gbest'ten daha iyiyse gbest i değiştir.
    if particle.fitness <= micpso_gbest:
        micpso_gbest = particle.fitness
        micpso_gbest_particle = particle
    else:
        micpso_gbest = micpso_gbest
        micpso_gbest_particle = micpso_gbest_particle

    print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6}".format(particle.particle_id, particle.ddrule,
                                                              particle.dsprule, str(
                                                                  particle.routes),
                                                              round(
                                                                  particle.fitness, 2),
                                                              round(particle.pbest, 2)))

print("-" * 100)
print(
    "gbest: {} gbest_particle: {}\n\n".format(round(micpso_gbest, 2), micpso_gbest_particle.particle_id))
print("-" * 100)

micpso_gbest_list.append(micpso_gbest)  # gbesti listeye ekle
micpso_gbest_particle_list.append(micpso_gbest_particle)

# başlangıç sürüsünü swarm liste kaydet
micpso_swarm_list.append(micpso_initial_swarm)


# iterasyonlara başla
for i in range(1, ps.iter_size):
    s = Swarm(size=pop_size, generation=i)
    print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6} swarm: {6:<4}".format("id", "dd", "dsp",
                                                                            "routes",
                                                                            "fitness", "pbest",
                                                                            str(s.generation)))
    print("-" * 100)

    for p, particle in enumerate(micpso_swarm_list[i-1].particles):
        new_particle = Chromosome()
        new_particle.particle_id = p

        r1 = np.random.random()
        r2 = np.random.random()

        dd_velocity = particle.ddrule_velocity[:]
        dsp_velocity = particle.dsprule_velocity[:]
        route_velocity = particle.routes_velocity[:]

        # new ddrule probs
        inertia_ddrule = [x * Swarm.w for x in dd_velocity]
        cognitive_ddrule = [r1 * Swarm.c_1 * (a + b) for a, b in
                            zip(particle.pbest_ddrule_probs, particle.ddrule_probs)]  # TODO
        social_ddrule = [r2 * Swarm.c_2 * (a + b) for a, b in
                         zip(micpso_gbest_particle.ddrule_probs, particle.ddrule_probs)]
        new_particle.ddrule_velocity = [a + b + c for a, b, c in
                                        zip(inertia_ddrule, cognitive_ddrule, social_ddrule)]
        new_ddrule_probs = [
            a + b for a, b in zip(particle.ddrule_probs, new_particle.ddrule_velocity)]
        # Toplamları 1 olacak şekilde oranladık.
        new_particle.ddrule_probs = [
            x / sum(new_ddrule_probs) for x in new_ddrule_probs]
        # new dsprule
        inertia_dsprule = [x * Swarm.w for x in dsp_velocity]
        cognitive_dsprule = [r1 * Swarm.c_1 * (a + b) for a, b in zip(
            particle.pbest_dsprule_probs, particle.dsprule_probs)]  # TODO
        social_dsprule = [r2 * Swarm.c_2 * (a + b) for a, b in
                          zip(micpso_gbest_particle.dsprule_probs, particle.dsprule_probs)]
        new_particle.dsprule_velocity = [a + b + c for a, b, c in
                                         zip(inertia_dsprule, cognitive_dsprule, social_dsprule)]
        new_dsprule_probs = [
            a + b for a, b in zip(particle.dsprule_probs, new_particle.dsprule_velocity)]
        new_particle.dsprule_probs = [
            x / sum(new_dsprule_probs) for x in new_dsprule_probs]

        # new routes
        new_particle.routes_probs = []
        new_particle.routes_velocity = []
        for j_s in range(ps.job_size): # TODO inertia için %25 rastgele mutasyon %25 geni pbest ile değiştir. %25 geni gbest ile değiştir.
            # New routes
            inertia_routes = [x * Swarm.w for x in route_velocity[j_s]]
            cognitive_routes = [r1 * Swarm.c_1 * (a + b) for a, b in
                                zip(particle.pbest_routes_probs[j_s], particle.routes_probs[j_s])]  # TODO
            social_routes = [r2 * Swarm.c_2 * (a + b) for a, b in
                             zip(micpso_gbest_particle.routes_probs[j_s], particle.routes_probs[j_s])]
            rv = [a + b + c for a, b, c in
                  zip(inertia_routes, cognitive_routes, social_routes)]
            new_particle.routes_velocity.append(rv)
            new_routes_probs = [a + b for a, b in
                                zip(particle.routes_probs[j_s], new_particle.routes_velocity[j_s])]
            normalized_routes_probs = [
                x / sum(new_routes_probs) for x in new_routes_probs]
            new_particle.routes_probs.append(normalized_routes_probs)

        new_particle.routes = []
        for i in range(ps.job_size):
            route = np.random.choice(Chromosome.routes_values[i], size=1, p=new_particle.routes_probs[i])[0]
            new_particle.routes.append(route)
        new_particle.ddrule = np.random.choice(Chromosome.ddrule_values, size=1, p=new_particle.ddrule_probs)[0]

        new_particle.dsprule = np.random.choice(Chromosome.dsprule_values, size=1, p=new_particle.dsprule_probs)[0]


        # Calculate new fitness
        new_particle.generate_data()
        new_particle.due_date_assignment()
        new_particle.run_simulation()
        new_particle.calculate_fitness()
        # print(p, ". ", "particle.fitness", particle.fitness)

        # Pbest
        if new_particle.fitness <= particle.pbest:
            new_particle.pbest = new_particle.fitness
            new_particle.pbest_ddrule_value = new_particle.ddrule
            new_particle.pbest_dsprule_value = new_particle.dsprule
            new_particle.pbest_routes_value = new_particle.routes[:]
            # probs
            new_particle.pbest_ddrule_probs = new_particle.ddrule_probs[:]
            new_particle.pbest_dsprule_probs = new_particle.dsprule_probs[:]
            new_particle.pbest_routes_probs = new_particle.routes_probs[:]
        else:
            # mutation
            if r1 < 0.6:
                new_particle.ddrule = micpso_gbest_particle.ddrule
            else:
                new_particle.ddrule = particle.pbest_ddrule_value
            if r2 < 0.6:
                new_particle.dsprule = micpso_gbest_particle.dsprule
            else:
                new_particle.dsprule = particle.pbest_dsprule_value
            if r2 < 0.7:
                crx_point = np.random.randint(0,ps.job_size,size=1)
                new_particle.routes = micpso_gbest_particle.routes[:crx_point[0]+1]+new_particle.routes[crx_point[0]+1:]
                mut_points = np.random.randint(0,ps.job_size,size=int(ps.job_size/10))
                for mut_point in mut_points:
                    new_particle.routes[mut_point]=micpso_gbest_particle.routes[mut_point]
            
            # calculate again
            new_particle.generate_data()
            new_particle.due_date_assignment()
            new_particle.run_simulation()
            new_particle.calculate_fitness()
            
            if new_particle.fitness <= particle.pbest:
                new_particle.pbest = new_particle.fitness
                new_particle.pbest_ddrule_value = new_particle.ddrule
                new_particle.pbest_dsprule_value = new_particle.dsprule
                new_particle.pbest_routes_value = new_particle.routes[:]
                # probs
                new_particle.pbest_ddrule_probs = new_particle.ddrule_probs[:]
                new_particle.pbest_dsprule_probs = new_particle.dsprule_probs[:]
                new_particle.pbest_routes_probs = new_particle.routes_probs[:]
            else:
                new_particle.pbest = particle.pbest
                new_particle.pbest_ddrule_value = particle.ddrule
                new_particle.pbest_dsprule_value = particle.dsprule
                new_particle.pbest_routes_value = particle.routes[:]
                # probs
                new_particle.pbest_ddrule_probs = particle.ddrule_probs[:]
                new_particle.pbest_dsprule_probs = particle.dsprule_probs[:]
                new_particle.pbest_routes_probs = particle.routes_probs[:]

            
        

        # Gbest
        if new_particle.fitness <= micpso_gbest:
            micpso_gbest = new_particle.fitness
            micpso_gbest_particle = new_particle
        else:
            micpso_gbest = micpso_gbest
            micpso_gbest_particle = micpso_gbest_particle

        print(particle.particle_id, particle.ddrule, particle.dsprule, str(
            particle.routes), round(particle.pbest, 2), round(particle.fitness, 2))
        s.particles.append(new_particle)

    print("-" * 100)
    print(
        "gbest: {} gbest_particle: {}\n\n".format(round(micpso_gbest, 2), micpso_gbest_particle.particle_id))
    print("-" * 100)
    
    micpso_gbest_list.append(micpso_gbest)
    micpso_gbest_particle_list.append(micpso_gbest_particle)
    # eğer gbest değiştiyse:

    micpso_swarm_list.append(s)

# çalıştırma zamanını kaydet
micpso_finish_time = time.time()

np.savetxt(fname='best_performances\\micpso_perf' +
           str(at_no)+'.txt', X=micpso_gbest_list, fmt='%1.3f')
np.savetxt(fname='best_performances\\micpso_CPU' +
           str(at_no)+'.txt', X=exec_time(micpso_start_time, micpso_finish_time))
micpso_genes = [micpso_gbest_particle.ddrule, micpso_gbest_particle.dsprule]+micpso_gbest_particle.routes
np.savetxt(fname='best_performances\\micpso_solution' +
           str(at_no)+'.txt', X=micpso_genes)

# MDPSO

In [ ]:
# MDPSO

# Magics
%precision 2

mdpso_start_time = time.time()
mdpso_swarm_list = []

mdpso_gbest = float('inf')
mdpso_gbest_list = []
mdpso_gbest_particle_list = []
mdpso_gbest_particle = None

# başlangıç sürüsü
print("-" * 100)
print("Initial Swarm", "w=", Swarm.w, "c1=", Swarm.c_1, "c2=", Swarm.c_2)
print("-" * 100)
print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6} {6:<10}".format("id", "dd", "dsp", "routes",
                                                                  "fitness", "pbest", "swarm: 0"))
print("-" * 100)
mdpso_initial_swarm = Swarm(size=pop_size, generation=0)
for i in range(pop_size):
    mdpso_initial_swarm.particles.append(Chromosome(particle_id=i))
for particle in mdpso_initial_swarm.particles:
    # Randomly generate first swarm
    # 0 ile 1 arasında ddrule size kadar rastgele.
    particle.ddrule_probs = sum_to_x(ddrule_size, sum_to)
    # particle.ddrule_probs = [(1 / ddrule_size) for i in range(ddrule_size)]
    particle.dsprule_probs = sum_to_x(dsprule_size, sum_to)
    # particle.dsprule_probs = [(1 / dsprule_size) for i in range(dsprule_size)]
    particle.routes_probs = []
    for i in range(ps.job_size):
        particle.routes_probs.append(sum_to_x(ps.route_size, sum_to))

    # Make sample -- Olasılıklardan değer seç
    particle.ddrule = np.random.choice(
        Chromosome.ddrule_values, 1, p=particle.ddrule_probs)[0]
    particle.dsprule = np.random.choice(
        Chromosome.dsprule_values, 1, p=particle.dsprule_probs)[0]
    particle.routes = []
    for i in range(ps.job_size):
        prob = particle.routes_probs[i]
        value = Chromosome.routes_values[i]
        route_value = np.random.choice(value, size=1, p=prob)[0]
        particle.routes.append(route_value)

    # Fitness değerlerini hesapla
    particle.set_positions()
    particle.change_genes(particle.position_values)
    particle.generate_data()
    particle.due_date_assignment()
    particle.run_simulation(yazdir=False)
    particle.calculate_fitness()
    particle.pbest = particle.fitness
    particle.pbest_ddrule_probs = particle.ddrule_probs[:]
    particle.pbest_dsprule_probs = particle.dsprule_probs[:]
    particle.pbest_routes_probs = particle.routes_probs[:]
    particle.pbest_ddrule_value = particle.ddrule
    particle.pbest_dsprule_value = particle.dsprule
    particle.pbest_routes_values = particle.routes
    particle.ddrule_velocity = [0] * ddrule_size
    particle.dsprule_velocity = [0] * dsprule_size
    particle.routes_velocity = []
    for i in range(ps.job_size):
        particle.routes_velocity.append([0] * ps.route_size)

    # eğer sürüdeki bir parçacık gbest'ten daha iyiyse gbest i değiştir.
    if particle.fitness <= mdpso_gbest:
        mdpso_gbest = particle.fitness
        mdpso_gbest_particle = particle
    else:
        mdpso_gbest = mdpso_gbest
        mdpso_gbest_particle = mdpso_gbest_particle

    print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6}".format(particle.particle_id, particle.ddrule,
                                                              particle.dsprule, str(
                                                                  particle.routes),
                                                              round(
                                                                  particle.fitness, 2),
                                                              round(particle.pbest, 2)))

print("-" * 100)
print(
    "gbest: {} gbest_particle: {}\n\n".format(round(mdpso_gbest, 2), mdpso_gbest_particle.particle_id))
print("-" * 100)

mdpso_gbest_list.append(mdpso_gbest)  # gbesti listeye ekle
mdpso_gbest_particle_list.append(mdpso_gbest_particle)

# başlangıç sürüsünü swarm liste kaydet
mdpso_swarm_list.append(mdpso_initial_swarm)


# iterasyonlara başla
for i in range(1, ps.iter_size):
    s = Swarm(size=pop_size, generation=i)
    print("{0:<5} {1:<5} {2:<5} {3:<30} {4:<6} {5:<6} swarm: {6:<4}".format("id", "dd", "dsp",
                                                                            "routes",
                                                                            "fitness", "pbest",
                                                                            str(s.generation)))
    print("-" * 100)

    for p, particle in enumerate(mdpso_swarm_list[i-1].particles):
        new_particle = Chromosome()
        new_particle.particle_id = p

        r1 = np.random.random()
        r2 = np.random.random()

        dd_velocity = particle.ddrule_velocity[:]
        dsp_velocity = particle.dsprule_velocity[:]
        route_velocity = particle.routes_velocity[:]

        # new ddrule probs
        inertia_ddrule = [x * Swarm.w for x in dd_velocity]
        cognitive_ddrule = [r1 * Swarm.c_1 * (a + b) for a, b in
                            zip(particle.pbest_ddrule_probs, particle.ddrule_probs)]  # TODO
        social_ddrule = [r2 * Swarm.c_2 * (a + b) for a, b in
                         zip(mdpso_gbest_particle.ddrule_probs, particle.ddrule_probs)]
        new_particle.ddrule_velocity = [a + b + c for a, b, c in
                                        zip(inertia_ddrule, cognitive_ddrule, social_ddrule)]
        new_ddrule_probs = [
            a + b for a, b in zip(particle.ddrule_probs, new_particle.ddrule_velocity)]
        # Toplamları 1 olacak şekilde oranladık.
        new_particle.ddrule_probs = [
            x / sum(new_ddrule_probs) for x in new_ddrule_probs]
        # new dsprule
        inertia_dsprule = [x * Swarm.w for x in dsp_velocity]
        cognitive_dsprule = [r1 * Swarm.c_1 * (a + b) for a, b in zip(
            particle.pbest_dsprule_probs, particle.dsprule_probs)]  # TODO
        social_dsprule = [r2 * Swarm.c_2 * (a + b) for a, b in
                          zip(mdpso_gbest_particle.dsprule_probs, particle.dsprule_probs)]
        new_particle.dsprule_velocity = [a + b + c for a, b, c in
                                         zip(inertia_dsprule, cognitive_dsprule, social_dsprule)]
        new_dsprule_probs = [
            a + b for a, b in zip(particle.dsprule_probs, new_particle.dsprule_velocity)]
        new_particle.dsprule_probs = [
            x / sum(new_dsprule_probs) for x in new_dsprule_probs]

        # new routes
        new_particle.routes_probs = []
        new_particle.routes_velocity = []
        for j_s in range(ps.job_size): # TODO inertia için %25 rastgele mutasyon %25 geni pbest ile değiştir. %25 geni gbest ile değiştir.
            # New routes
            inertia_routes = [x * Swarm.w for x in route_velocity[j_s]]
            cognitive_routes = [r1 * Swarm.c_1 * (a + b) for a, b in
                                zip(particle.pbest_routes_probs[j_s], particle.routes_probs[j_s])]  # TODO
            social_routes = [r2 * Swarm.c_2 * (a + b) for a, b in
                             zip(mdpso_gbest_particle.routes_probs[j_s], particle.routes_probs[j_s])]
            rv = [a + b + c for a, b, c in
                  zip(inertia_routes, cognitive_routes, social_routes)]
            new_particle.routes_velocity.append(rv)
            new_routes_probs = [a + b for a, b in
                                zip(particle.routes_probs[j_s], new_particle.routes_velocity[j_s])]
            normalized_routes_probs = [
                x / sum(new_routes_probs) for x in new_routes_probs]
            new_particle.routes_probs.append(normalized_routes_probs)

        new_particle.routes = []
        for i in range(ps.job_size):
            route = np.random.choice(Chromosome.routes_values[i], size=1, p=new_particle.routes_probs[i])[0]
            new_particle.routes.append(route)
        new_particle.ddrule = np.random.choice(Chromosome.ddrule_values, size=1, p=new_particle.ddrule_probs)[0]

        new_particle.dsprule = np.random.choice(Chromosome.dsprule_values, size=1, p=new_particle.dsprule_probs)[0]


        # Calculate new fitness
        new_particle.generate_data()
        new_particle.due_date_assignment()
        new_particle.run_simulation()
        new_particle.calculate_fitness()
        # print(p, ". ", "particle.fitness", particle.fitness)

        # Pbest
        if new_particle.fitness <= particle.pbest:
            new_particle.pbest = new_particle.fitness
            new_particle.pbest_ddrule_value = new_particle.ddrule
            new_particle.pbest_dsprule_value = new_particle.dsprule
            new_particle.pbest_routes_value = new_particle.routes[:]
            # probs
            new_particle.pbest_ddrule_probs = new_particle.ddrule_probs[:]
            new_particle.pbest_dsprule_probs = new_particle.dsprule_probs[:]
            new_particle.pbest_routes_probs = new_particle.routes_probs[:]
        else:
            # inertia %25 geni mutasyona uğrat
            import random
            gene_size = ps.job_size+2
            mutation_points = random.sample(list(range(gene_size)),int(gene_size*0.25))
            
            for mut_point in mutation_points:
                if mut_point == 0:
                    new_particle.ddrule = np.random.randint(0,ddrule_size)
                elif mut_point == 1:
                    new_particle.dsprule = np.random.randint(0,dsprule_size)
                else:
                    degisecek_gene = mut_point-2
                    new_particle.routes[degisecek_gene] = np.random.randint(0,ps.route_size)
                
            # cognitive %25 geni pbest ile değiştir
            pbest_crossover_points = random.sample(list(range(gene_size)),int(gene_size*0.25))
            for pbest_cro in pbest_crossover_points:
                if pbest_cro == 0:
                    new_particle.ddrule = new_particle.pbest_ddrule_value
                elif pbest_cro == 1:
                    new_particle.dsprule = new_particle.pbest_dsprule_value
                else:
                    degisecek_gene = mut_point-2
                    new_particle.routes[degisecek_gene] = mdpso_gbest_particle.routes[degisecek_gene]
            # social %25 geni gbest ile değiştir
            gbest_crossover_points = random.sample(list(range(gene_size)),int(gene_size*0.25))
            for gbest_cro in gbest_crossover_points:
                if gbest_cro == 0:
                    new_particle.ddrule =mdpso_gbest_particle.ddrule
                elif gbest_cro == 1:
                    new_particle.dsprule =mdpso_gbest_particle.dsprule
                else:
                    degisecek_gene = mut_point-2
                    new_particle.routes[degisecek_gene] = mdpso_gbest_particle.routes[degisecek_gene]
            
            # calculate again
            new_particle.generate_data()
            new_particle.due_date_assignment()
            new_particle.run_simulation()
            new_particle.calculate_fitness()
            
            if new_particle.fitness <= particle.pbest:
                new_particle.pbest = new_particle.fitness
                new_particle.pbest_ddrule_value = new_particle.ddrule
                new_particle.pbest_dsprule_value = new_particle.dsprule
                new_particle.pbest_routes_value = new_particle.routes[:]
                # probs
                new_particle.pbest_ddrule_probs = new_particle.ddrule_probs[:]
                new_particle.pbest_dsprule_probs = new_particle.dsprule_probs[:]
                new_particle.pbest_routes_probs = new_particle.routes_probs[:]
            else:
                new_particle.pbest = particle.pbest
                new_particle.pbest_ddrule_value = particle.ddrule
                new_particle.pbest_dsprule_value = particle.dsprule
                new_particle.pbest_routes_value = particle.routes[:]
                # probs
                new_particle.pbest_ddrule_probs = particle.ddrule_probs[:]
                new_particle.pbest_dsprule_probs = particle.dsprule_probs[:]
                new_particle.pbest_routes_probs = particle.routes_probs[:]

            
        

        # Gbest
        if new_particle.fitness <= mdpso_gbest:
            mdpso_gbest = new_particle.fitness
            mdpso_gbest_particle = new_particle
        else:
            mdpso_gbest = mdpso_gbest
            mdpso_gbest_particle = mdpso_gbest_particle

        print(particle.particle_id, particle.ddrule, particle.dsprule, str(
            particle.routes), round(particle.pbest, 2), round(particle.fitness, 2))
        s.particles.append(new_particle)

    print("-" * 100)
    print(
        "gbest: {} gbest_particle: {}\n\n".format(round(mdpso_gbest, 2), mdpso_gbest_particle.particle_id))
    print("-" * 100)
    
    mdpso_gbest_list.append(mdpso_gbest)
    mdpso_gbest_particle_list.append(mdpso_gbest_particle)
    # eğer gbest değiştiyse:

    mdpso_swarm_list.append(s)

# çalıştırma zamanını kaydet
mdpso_finish_time = time.time()
np.savetxt(fname='best_performances\\mdpso_perf' +
           str(at_no)+'.txt', X=mdpso_gbest_list, fmt='%1.3f')
np.savetxt(fname='best_performances\\mdpso_CPU' +
           str(at_no)+'.txt', X=exec_time(mdpso_start_time, mdpso_finish_time))
mdpso_genes = [mdpso_gbest_particle.ddrule, mdpso_gbest_particle.dsprule]+mdpso_gbest_particle.routes
np.savetxt(fname='best_performances\\mdpso_solution' +
           str(at_no)+'.txt', X=mdpso_genes)

# Results

In [ ]:
# Algoritmalar içerisinde en iyi sonucu veren algoritmayı bul
best_chromosomes = {"GA":pop_list[-1].chromosomes[0], "SA":solution_x, "TA":tabu_solution_x,"GASA":gasa_solution_x,"GATA":gata_solution_x}
optimum_performances = {"GA":min(ga_perf),"SA":min(sa_perf),"TA":min(ta_perf),"GASA":min(gasa_perf),"GATA":min(gata_perf)}
optimum_algorithm = min(optimum_performances,key=optimum_performances.get)
optimum_chromosome=best_chromosomes[optimum_algorithm]
best_algorithm={}
best_algorithm['algorithm']=optimum_algorithm
best_algorithm['fitness']=min(list(optimum_performances.values()))
best_algorithm['genes']=str(optimum_chromosome.genes)
path='./'
fileName='best_algorithm'+str(at_no)

writeToJSONFile(path,fileName,best_algorithm)

In [ ]:
# Print results in a dataframe
chromosome = pop_list[-1].chromosomes[0]
# print("-"*25)
# print("Fitness", chromosome.fitness, "Route", chromosome.routes)
# print("Due Date Rule", chromosome.ddrule,
#       "Dispatchine Rule", chromosome.dsprule)
# print("-"*25)
#file_opt = open('optimum'+str(at_no)+'.obj', 'w') 
#pickle.dump(optimum_chromosome, file_opt)

job_no_list=[i for i, job in enumerate(chromosome.env.joblist)]
weight_list=[job.weight for i, job in enumerate(chromosome.env.joblist)]
arrival_time_list=[job.arrival_time for i,
    job in enumerate(chromosome.env.joblist)]
departure_time_list=[job.departure_time for i,
    job in enumerate(chromosome.env.joblist)]
given_due_date_list=[job.due_date_time for i,
    job in enumerate(chromosome.env.joblist)]
earliness_list=[]
tardiness_list=[]
penalty_earliness_list=[]
penalty_tardiness_list=[]
penalty_due_dates_list=[]
penalty_total_list=[]


total_list=[]
for i, job in enumerate(chromosome.env.joblist):

    tardiness=max(job.departure_time-job.due_date_time, 0)
    earliness=max(job.due_date_time-job.departure_time, 0)
    penalty_dd=job.weight *((8/480)*(job.due_date_time - job.arrival_time))
    if tardiness != 0:
        penalty_t=job.weight * (6+6*(tardiness/480))
    else:
        penalty_t=0
    if earliness != 0:
        penalty_e=job.weight * (5+4*(earliness/480))
    else:
        penalty_e=0
    penalty_total=penalty_dd+penalty_e+penalty_t
    earliness_list.append(earliness)
    tardiness_list.append(tardiness)
    penalty_earliness_list.append(penalty_e)
    penalty_tardiness_list.append(penalty_t)
    penalty_due_dates_list.append(penalty_dd)
    penalty_total_list.append(penalty_total)
    operation_no_list = []
    operation_duration_list=[]
    operation_machine_list=[]
    start_time_list=[]
    finish_time_list=[]
    job_operation = []
    
    
    for k, operation in enumerate(job.operation_list):
        new_list=[]
        new_list.append(i)
        new_list.append(k)
        new_list.append(operation.duration)
        new_list.append(operation.machine.mc_id)
        new_list.append(operation.start_time)
        new_list.append(operation.finish_time)
#         print(operation.name(), " start_time=",
#               operation.start_time, " finish_time=",
#               operation.finish_time)
        total_list.append(new_list)
    #print("-"*25)

# işlere ait bilgiler tutuluyor
df = pd.DataFrame({"weight": weight_list,
                   "arrival_time(s)":arrival_time_list, 
                   "departure_time(s)":departure_time_list, 
                   "given_due-date(s)":given_due_date_list, 
                   "earliness":earliness_list, 
                   "tardiness":tardiness_list, 
                   "penalty_earliness":penalty_earliness_list, 
                   "penalty_tardiness":penalty_tardiness_list, 
                   "penalty_due-dates": penalty_due_dates_list,
                   "penalty_total": penalty_total_list},index=job_no_list)


df2=pd.DataFrame(columns=["job_no",
                      "operation_no",
                  "duration(s)",
                  "assigned_machine",
                  "start(s)",
                  "finish(s)"])

for operation_list in total_list:
    df2.loc[-1]=operation_list
    df2.index=df2.index+1
df2.sort_index()

# Round each number in a Python pandas data frame by 2 decimals
df = df.round(2)
df2 = df2.round(2)
df.index.name="job"
df2.index.name="index"

#df.to_csv("results\\ET-results-"+str(at_no)+".csv")
#df2.to_csv("results\\scheduling-results-"+str(at_no)+".csv",index=False)

# to excel
writer = pd.ExcelWriter("results\\results-"+str(at_no)+".xlsx")
df.to_excel(writer,sheet_name='fitness_results')
df2.to_excel(writer,sheet_name='schedules',index=False)
writer.save()

In [ ]:
# Tüm atölyeler için performans grafiği çizdir.
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

fig = plt.figure()
shopfloor=at_no
#GA
# Tüm değerler
ga_perfs=np.loadtxt(fname='best_performances\\ga_perf'+str(shopfloor)+'.txt')

# En iyi değer
ga_best=min(ga_perfs)
ga_sorted=np.sort(ga_perfs)
#Ortalama değer
ga_avg=np.mean(ga_perfs)
#En kötü değer
ga_10_worst=ga_sorted[-2:]
ga_worst=np.average(ga_10_worst)

#SA

sa_perfs=np.loadtxt(fname='best_performances\\sa_perf'+str(shopfloor)+'.txt')
# En iyi değer
sa_best=min(sa_perfs)
sa_sorted=np.sort(sa_perfs)
#Ortalama değer
sa_avg=np.average(sa_perfs)
#En kötü değer
sa_10_worst=sa_sorted[-20:]
sa_worst=np.average(sa_10_worst)

#TA

ta_perfs=np.loadtxt(fname='best_performances\\ta_perf'+str(shopfloor)+'.txt')
# En iyi değer
ta_best=min(ta_perfs)
#Ortalama değer
ta_avg=np.mean(ta_perfs)
#En kötü değer
ta_sorted=np.sort(ta_perfs)
ta_10_worst=ta_sorted[-2:]
ta_worst=np.average(ta_10_worst)

#GASA

gasa_perfs=np.loadtxt(fname='best_performances\\gasa_perf'+str(shopfloor)+'.txt')
# En iyi değer
gasa_best=min(gasa_perfs)
#Ortalama değer
gasa_avg=np.mean(gasa_perfs)
#En kötü değer
gasa_sorted=np.sort(gasa_perfs)
gasa_10_worst=gasa_sorted[-2:]
gasa_worst=np.average(gasa_10_worst)

#GATA

gata_perfs=np.loadtxt(fname='best_performances\\gata_perf'+str(shopfloor)+'.txt')
# En iyi değer
gata_best=min(gata_perfs)
#Ortalama değer
gata_avg=np.mean(gata_perfs)
#En kötü değer
gata_sorted=np.sort(gata_perfs)
gata_10_worst=gata_sorted[-2:]
gata_worst=np.average(gata_10_worst)

plt.subplot(1,1,1)
plt.plot(['best', 'avg', 'worst'], [ga_best, ga_avg, ga_worst], label='GA', marker='x')
plt.plot(['best', 'avg', 'worst'], [sa_best, sa_avg, sa_worst], label='SA',marker='x')
plt.plot(['best', 'avg', 'worst'], [ta_best, ta_avg, ta_worst], label='TA',marker='x')
plt.plot(['best', 'avg', 'worst'], [gasa_best, gasa_avg, gasa_worst], label=r'GA\SA',marker='x')
plt.plot(['best', 'avg', 'worst'], [gata_best, gata_avg, gata_worst], label=r'GA\TA',marker='x')
plt.legend()
plt.grid(True)
plt.ylabel('Performance')
plt.title('Shop Floor'+str(at_no))
plt.savefig("graphics\\"+str(at_no)+'-comparision.svg')

In [ ]:
# Gantt Şeması

# Colors
cnames = {
    'black':                '#000000',
    'blue':                 '#0000FF',
    'blueviolet':           '#8A2BE2',
    'brown':                '#A52A2A',
    'burlywood':            '#DEB887',
    'cadetblue':            '#5F9EA0',
    'chartreuse':           '#7FFF00',
    'chocolate':            '#D2691E',
    'coral':                '#FF7F50',
    'cornflowerblue':       '#6495ED',
    'crimson':              '#DC143C',
    'cyan':                 '#00FFFF',
    'darkblue':             '#00008B',
    'darkcyan':             '#008B8B',
    'darkgoldenrod':        '#B8860B',
    'darkgray':             '#A9A9A9',
    'darkgreen':            '#006400',
    'darkkhaki':            '#BDB76B',
    'darkmagenta':          '#8B008B',
    'darkolivegreen':       '#556B2F',
    'darkorange':           '#FF8C00',
    'darkorchid':           '#9932CC',
    'darkred':              '#8B0000',
    'darksalmon':           '#E9967A',
    'darkseagreen':         '#8FBC8F',
    'darkslateblue':        '#483D8B',
    'darkslategray':        '#2F4F4F',
    'darkturquoise':        '#00CED1',
    'darkviolet':           '#9400D3',
    'deeppink':             '#FF1493',
    'deepskyblue':          '#00BFFF',
    'dimgray':              '#696969',
    'dodgerblue':           '#1E90FF',
    'firebrick':            '#B22222',
    'floralwhite':          '#FFFAF0',
    'forestgreen':          '#228B22',
    'fuchsia':              '#FF00FF',
    'gainsboro':            '#DCDCDC',
    'ghostwhite':           '#F8F8FF',
    'gold':                 '#FFD700',
    'goldenrod':            '#DAA520',
    'gray':                 '#808080',
    'green':                '#008000',
    'greenyellow':          '#ADFF2F',
    'honeydew':             '#F0FFF0',
    'hotpink':              '#FF69B4',
    'indianred':            '#CD5C5C',
    'indigo':               '#4B0082',
    'ivory':                '#FFFFF0',
    'khaki':                '#F0E68C',
    'lavender':             '#E6E6FA',
    'lavenderblush':        '#FFF0F5',
    'lawngreen':            '#7CFC00',
    'lemonchiffon':         '#FFFACD',
    'lightblue':            '#ADD8E6',
    'lightcoral':           '#F08080',
    'lightcyan':            '#E0FFFF',
    'lightgoldenrodyellow': '#FAFAD2',
    'lightgreen':           '#90EE90',
    'lightgray':            '#D3D3D3',
    'lightpink':            '#FFB6C1',
    'lightsalmon':          '#FFA07A',
    'lightseagreen':        '#20B2AA',
    'lightskyblue':         '#87CEFA',
    'lightslategray':       '#778899',
    'lightsteelblue':       '#B0C4DE',
    'lightyellow':          '#FFFFE0',
    'lime':                 '#00FF00',
    'limegreen':            '#32CD32',
    'linen':                '#FAF0E6',
    'magenta':              '#FF00FF',
    'maroon':               '#800000',
    'mediumaquamarine':     '#66CDAA',
    'mediumblue':           '#0000CD',
    'mediumorchid':         '#BA55D3',
    'mediumpurple':         '#9370DB',
    'mediumseagreen':       '#3CB371',
    'mediumslateblue':      '#7B68EE',
    'mediumspringgreen':    '#00FA9A',
    'mediumturquoise':      '#48D1CC',
    'mediumvioletred':      '#C71585',
    'midnightblue':         '#191970',
    'mintcream':            '#F5FFFA',
    'mistyrose':            '#FFE4E1',
    'moccasin':             '#FFE4B5',
    'navajowhite':          '#FFDEAD',
    'navy':                 '#000080',
    'oldlace':              '#FDF5E6',
    'olive':                '#808000',
    'olivedrab':            '#6B8E23',
    'orange':               '#FFA500',
    'orangered':            '#FF4500',
    'orchid':               '#DA70D6',
    'palegoldenrod':        '#EEE8AA',
    'palegreen':            '#98FB98',
    'paleturquoise':        '#AFEEEE',
    'palevioletred':        '#DB7093',
    'papayawhip':           '#FFEFD5',
    'peachpuff':            '#FFDAB9',
    'peru':                 '#CD853F',
    'pink':                 '#FFC0CB',
    'plum':                 '#DDA0DD',
    'powderblue':           '#B0E0E6',
    'purple':               '#800080',
    'red':                  '#FF0000',
    'rosybrown':            '#BC8F8F',
    'royalblue':            '#4169E1',
    'saddlebrown':          '#8B4513',
    'salmon':               '#FA8072',
    'sandybrown':           '#FAA460',
    'seagreen':             '#2E8B57',
    'seashell':             '#FFF5EE',
    'sienna':               '#A0522D',
    'silver':               '#C0C0C0',
    'skyblue':              '#87CEEB',
    'slateblue':            '#6A5ACD',
    'slategray':            '#708090',
    'snow':                 '#FFFAFA',
    'springgreen':          '#00FF7F',
    'steelblue':            '#4682B4',
    'tan':                  '#D2B48C',
    'teal':                 '#008080',
    'thistle':              '#D8BFD8',
    'tomato':               '#FF6347',
    'turquoise':            '#40E0D0',
    'violet':               '#EE82EE',
    'wheat':                '#F5DEB3',
    'white':                '#FFFFFF',
    'whitesmoke':           '#F5F5F5',
    'yellow':               '#FFFF00',
    'yellowgreen':          '#9ACD32'}
colors_list = list(cnames.values())


def gantt_cizdir(chromosome=optimum_chromosome):
    fig, ax = plt.subplots()
    cmap = cm.ScalarMappable(col.Normalize(0, 99), cm.hsv)
    # colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

    job_list = chromosome.env.joblist
    machine_list = chromosome.env.machinelist
    for i, job in enumerate(job_list):
        job.color = colors_list[i]

    departures = []
    # add tasks to the machines
    for job in job_list:
        departures.append(job.departure_time)
        for op in job.operation_list:
            op.machine.op_list.append(op)

    departures.sort()

    #ax.set_xlabel('seconds', fontsize=8)
    ax.set_ylabel('Machines', fontsize=8)
    ax.set_yticks(list(range(10, 15 * len(machine_list) + 10, 15)))
    ax.set_yticklabels(['m' + str(i + 1)
                        for i in range(len(machine_list))], fontsize=8)
    ax.set_xticks(
        list(range(0, departures[-1] + 20, int(departures[-1] / 10))))
    ax.grid(color='lightblue', linestyle='-.', linewidth=0.1)

    for i, machine in enumerate(machine_list):
        xranges = [(op.start_time, op.duration) for op in machine.op_list]
        yrange = ((i * 15 + 5), 10)
        colors = [op.job.color for op in machine.op_list]
        ax.broken_barh(xranges, yrange, facecolors=colors,
                       edgecolor="black", linewidth=0.25)
    for job in job_list:
        ax.annotate('job arrival', (job.arrival_time, (job.operation_list[0].machine.mc_id)*15+5),
                    xytext=(0.5, 0.5), textcoords='axes fraction',
                    arrowprops={'facecolor': 'red', 'shrink': 0.002, 'width': 0.1,
                                'headwidth': 5, 'headlength': 10, 'alpha': 0.4},
                    fontsize=10,
                    horizontalalignment='right', verticalalignment='bottom')
    legend_elements = [plt.Line2D(
        [0], [0], color=job.color, label=job) for job in job_list]
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5, edgecolor='b', fontsize=8)
    plt.grid(True)
    plt.tight_layout()
    plt.title('Shop floor '+str(at_no), fontsize=10)
    plt.savefig(fname="graphics\\gantt_job_" + str(ps.job_size) + "_" + str(ps.machine_size) + "_machines.svg", dpi=300,
                facecolor='w', edgecolor='b', orientation='portrait', papertype=None,
                format=None, transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=None)

    plt.show()

if at_no < 3:
    gantt_cizdir()
else:
    pass

In [ ]:
# 8. floor average result
import matplotlib.pyplot as plt
import numpy as np
at_no=8
ga_81 = np.loadtxt(fname='best_performances\\ga_perf81.txt')
ga_82 = np.loadtxt(fname='best_performances\\ga_perf82.txt')
ga_83 = np.loadtxt(fname='best_performances\\ga_perf83.txt')
ga_84 = np.loadtxt(fname='best_performances\\ga_perf84.txt')
ga_85 = np.loadtxt(fname='best_performances\\ga_perf85.txt')

sa_81 = np.loadtxt(fname='best_performances\\sa_perf81.txt')
sa_82 = np.loadtxt(fname='best_performances\\sa_perf82.txt')
sa_83 = np.loadtxt(fname='best_performances\\sa_perf83.txt')
sa_84 = np.loadtxt(fname='best_performances\\sa_perf84.txt')
sa_85 = np.loadtxt(fname='best_performances\\sa_perf85.txt')

ta_81 = np.loadtxt(fname='best_performances\\ta_perf81.txt')
ta_82 = np.loadtxt(fname='best_performances\\ta_perf82.txt')
ta_83 = np.loadtxt(fname='best_performances\\ta_perf83.txt')
ta_84 = np.loadtxt(fname='best_performances\\ta_perf84.txt')
ta_85 = np.loadtxt(fname='best_performances\\ta_perf85.txt')

gasa_81 = np.loadtxt(fname='best_performances\\gasa_perf81.txt')
gasa_82 = np.loadtxt(fname='best_performances\\gasa_perf82.txt')
gasa_83 = np.loadtxt(fname='best_performances\\gasa_perf83.txt')
gasa_84 = np.loadtxt(fname='best_performances\\gasa_perf84.txt')
gasa_85 = np.loadtxt(fname='best_performances\\gasa_perf85.txt')

gata_81 = np.loadtxt(fname='best_performances\\gata_perf81.txt')
gata_82 = np.loadtxt(fname='best_performances\\gata_perf82.txt')
gata_83 = np.loadtxt(fname='best_performances\\gata_perf83.txt')
gata_84 = np.loadtxt(fname='best_performances\\gata_perf84.txt')
gata_85 = np.loadtxt(fname='best_performances\\gata_perf85.txt')


ga_8avg = [(x1+x2+x3+x4+x5)/5 for x1,x2,x3,x4,x5 in zip(ga_81,ga_82,ga_83,ga_84,ga_85)]
sa_8avg = [(x1+x2+x3+x4+x5)/5 for x1,x2,x3,x4,x5 in zip(sa_81,sa_82,sa_83,sa_84,sa_85)]
ta_8avg = [(x1+x2+x3+x4+x5)/5 for x1,x2,x3,x4,x5 in zip(ta_81,ta_82,ta_83,ta_84,ta_85)]
gasa_8avg = [(x1+x2+x3+x4+x5)/5 for x1,x2,x3,x4,x5 in zip(gasa_81,gasa_82,gasa_83,gasa_84,gasa_85)]
gata_8avg = [(x1+x2+x3+x4+x5)/5 for x1,x2,x3,x4,x5 in zip(gata_81,gata_82,gata_83,gata_84,gata_85)]


# En iyi değer
ga_best=min(ga_8avg)
ga_sorted=np.sort(ga_8avg)
#Ortalama değer
ga_avg=np.mean(ga_8avg)
#En kötü değer
ga_10_worst=ga_sorted[-2:]
ga_worst=np.average(ga_10_worst)

#SA

# En iyi değer
sa_best=min(sa_8avg)
sa_sorted=np.sort(sa_8avg)
#Ortalama değer
sa_avg=np.average(sa_8avg)
#En kötü değer
sa_10_worst=sa_sorted[-20:]
sa_worst=np.average(sa_10_worst)

#TA

# En iyi değer
ta_best=min(ta_8avg)
#Ortalama değer
ta_avg=np.mean(ta_8avg)
#En kötü değer
ta_sorted=np.sort(ta_8avg)
ta_10_worst=ta_sorted[-2:]
ta_worst=np.average(ta_10_worst)

#GASA

# En iyi değer
gasa_best=min(gasa_8avg)
#Ortalama değer
gasa_avg=np.mean(gasa_8avg)
#En kötü değer
gasa_sorted=np.sort(gasa_8avg)
gasa_10_worst=gasa_sorted[-2:]
gasa_worst=np.average(gasa_10_worst)

#GATA

# En iyi değer
gata_best=min(gata_8avg)
#Ortalama değer
gata_avg=np.mean(gata_8avg)
#En kötü değer
gata_sorted=np.sort(gata_8avg)
gata_10_worst=gata_sorted[-2:]
gata_worst=np.average(gata_10_worst)

plt.subplot(1,1,1)
plt.plot(['best', 'avg', 'worst'], [ga_best, ga_avg, ga_worst], label='GA', marker='x')
plt.plot(['best', 'avg', 'worst'], [sa_best, sa_avg, sa_worst], label='SA',marker='x')
plt.plot(['best', 'avg', 'worst'], [ta_best, ta_avg, ta_worst], label='TA',marker='x')
plt.plot(['best', 'avg', 'worst'], [gasa_best, gasa_avg, gasa_worst], label=r'GA\SA',marker='x')
plt.plot(['best', 'avg', 'worst'], [gata_best, gata_avg, gata_worst], label=r'GA\TA',marker='x')
plt.legend()
plt.grid(True)
plt.ylabel('Performance')
plt.title('Shop Floor'+str(at_no))
plt.savefig("graphics\\"+str(at_no)+'avg-comparision.svg')

# Özel Tanımlanmış Fonksiyonlar
__trace_schedule()__: Yapılan simülasyonun adımlarını gösterir.<br>
__print_schedule()__: Operasyonların hangi makinelerde hangi zamanlarda yer aldığını gösterir.<br>
__gantt_cizdir()__: Atamaların gantt şemasını çıkartır.<br>
__performans_cizdir()__: Algoritmanın sonuçlarını gösterir.

In [ ]:
# fonksiyonlar


def trace_schedule(genes=optimum_chromosome.genes):
    chr = Chromosome()
    chr.change_genes(genes)
    chr.generate_data()
    chr.due_date_assignment()
    chr.run_simulation(yazdir=True)
    chr.calculate_fitness()


def print_schedule(chromosome=optimum_chromosome):
    print("--------------------------------------------------")
    print("Chromosome",
          chromosome.chr_id, "Fitness", chromosome.fitness, "Route", chromosome.routes)
    print("Due Date Rule", chromosome.ddrule,
          "Dispatchine Rule", chromosome.dsprule)
    print("--------------------------------------------------")
    for i, job in enumerate(chromosome.env.joblist):
        print("job", i, " arrival_time= ", job.arrival_time, "departure_time=", job.departure_time, "due-date=",
              job.due_date_time, "weight= ", job.weight)
        for k, operation in enumerate(job.operation_list):
            print(operation.name(), " start_time=",
                  operation.start_time, " finish_time=",
                  operation.finish_time)
        print("--------------------------------------------------")


def gantt_cizdir(chromosome=optimum_chromosome):
    fig, ax = plt.subplots()
    cmap = cm.ScalarMappable(col.Normalize(0, 99), cm.hsv)
    # colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

    job_list = chromosome.env.joblist
    machine_list = chromosome.env.machinelist
    for job in job_list:
        job.color = cmap.to_rgba(np.random.randint(99))

    departures = []
    # add tasks to the machines
    for job in job_list:
        departures.append(job.departure_time)
        for op in job.operation_list:
            op.machine.op_list.append(op)

    departures.sort()

    ax.set_xlabel('seconds')
    ax.set_ylabel('Machines')
    ax.set_yticks(list(range(10, 15 * len(machine_list) + 10, 15)))
    ax.set_yticklabels(['m' + str(i + 1) for i in range(len(machine_list))])
    ax.set_xticks(
        list(range(0, departures[-1] + 20, int(departures[-1] / 10))))
    ax.grid(color='lightblue', linestyle='-.', linewidth=0.1)

    for i, machine in enumerate(machine_list):
        xranges = [(op.start_time, op.duration) for op in machine.op_list]
        yrange = ((i * 15 + 5), 10)
        colors = [op.job.color for op in machine.op_list]
        ax.broken_barh(xranges, yrange, facecolors=colors,
                       edgecolor="black", linewidth=0.25)
    for job in job_list:
        ax.annotate('job arrival', (job.arrival_time, (job.operation_list[0].machine.mc_id)*15+5),
                    xytext=(0.5, 0.5), textcoords='axes fraction',
                    arrowprops={'facecolor': 'red', 'shrink': 0.002, 'width': 0.1,
                                'headwidth': 5, 'headlength': 10, 'alpha': 0.4},
                    fontsize=10,
                    horizontalalignment='right', verticalalignment='bottom')
    # for job in job_list:
    #   ax.annotate('job'+ 'arrives', (job.arrival_time, (job.operation_list[0].machine.mc_id)*15+5),
    #           xytext=(0.89, 0.97), textcoords='axes fraction',
    #               arrowprops={'facecolor': 'black', 'shrink': 0.002, 'width': 0.1,'headwidth':5, 'headlength': 10},
    #           fontsize=10,
    #           horizontalalignment='right', verticalalignment='top')
    legend_elements = [plt.Line2D(
        [0], [0], color=job.color, label=job) for job in job_list]
    ax.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5, edgecolor='b')
    plt.tight_layout()
    plt.title(str(ps.job_size) + ' jobs, ' + str(ps.operation_size) + ' operations, ' + str(
        ps.machine_size) + ' machines, ' + str(ps.route_size) + ' routes')
    plt.savefig(fname="gantt_job_" + str(ps.job_size) + "_" + str(ps.machine_size) + "_machines.svg", dpi=300,
                facecolor='w', edgecolor='b', orientation='portrait', papertype=None,
                format=None, transparent=False, bbox_inches='tight', pad_inches=0.1, frameon=None)
    plt.show()


def performans_cizdir():
    bests = []
    avgs = []
    worsts = []

    for pop in pop_list[3:]:
        bests.append(pop.best)
        avgs.append(pop.avg)
        worsts.append(pop.worst)

    fig, ax = plt.subplots()
    ax.plot(bests, label='Best', marker='P')
    ax.plot(avgs, label='Avg', marker='o')
    ax.plot(worsts, label='Worst', marker='s')
    ax.set_xlabel('Generation')
    ax.set_ylabel('Performance')

    ax.legend(loc='upper right', shadow=True, fontsize='large')
    plt.tight_layout()
    plt.title(str(ps.job_size) + ' jobs, ' + str(ps.operation_size) + ' operations, ' + str(
        ps.machine_size) + ' machines, ' + str(ps.route_size) + ' routes')
    plt.savefig(fname="perf_job" + str(ps.job_size) + "_" + str(ps.machine_size) + "_machines.svg", dpi=300,
                facecolor='w',
                edgecolor='w', orientation='portrait', papertype=None,
                format=None, transparent=False, bbox_inches=None, pad_inches=0.1, frameon=None)
    plt.grid(color='lightblue', linestyle='-.', linewidth=0.6)
    plt.show()
    


In [ ]:
# Fonsiyonların test alanı

#trace_schedule()
#print_schedule()
#gantt_cizdir() 
#performans_cizdir()